## 5. Advanced RAG 기법

> **Advanced RAG는 Naive RAG의 단순한 검색 → 생성 구조를 확장하여,  
> 검색 품질을 정교하게 향상시키고, 문맥 전달·정렬·필터링·압축을 최적화하여  
> LLM이 더 정확한 근거 기반 응답을 생성하도록 설계된 RAG 기법들의 집합입니다.**

<img src="image/paradigms_of_RAG.png" width="600">

### 5.1 고급 Chunking 기법

RAG에서 **가장 먼저 손봐야 하는 곳**이 바로 Chunking입니다.  
동일한 임베딩 모델·벡터DB·LLM을 쓰더라도, **어떻게 문서를 쪼개느냐에 따라 검색 품질이 극단적으로 달라집니다.**

이 절에서는 다음 세 가지를 다룹니다.

1. 고정 길이 청크 (Fixed-size Chunking)  
2. 재귀적 청크 (Recursive Text Splitting)  
3. 구조 기반 청크 (Structure-aware Chunking)  

#### 5.1.1 왜 Chunking이 중요한가?

임베딩 모델과 벡터DB는 **청크 단위로 검색**을 한다.

- 벡터DB에 저장되는 기본 단위: **청크(chunk)**  
- 검색은 문서 전체가 아니라 청크 벡터들을 대상으로 수행  
- 따라서,
  - 청크가 너무 길면 → 하나의 청크 안에 여러 주제가 섞여 **노이즈 증가**  
  - 청크가 너무 짧으면 → 의미 단위가 깨져서 **문맥 정보 손실**

**좋은 청크의 조건**

1. **완결된 의미 단위**여야 한다. (문단·섹션 단위 등)  
2. **너무 길지도, 너무 짧지도 않아야** 한다. (보통 200~800 tokens 권장)  
3. 서로 **적당히 겹침(overlap)** 이 있어 컨텍스트를 안정적으로 이어줌

이제 각각의 대표적인 청크 방식과 실습 코드를 보겠습니다.

#### 5.1.2 고정 길이 청크 (Fixed-size Chunking)

가장 단순하고 이해하기 쉬운 방식입니다.

- 일정 길이(문자 수 또는 토큰 수) 기준으로 **앞에서부터 잘라 나가는 방식**
- 장점
  - 구현이 매우 쉬움
  - 데이터 형식과 상관없이 동일한 전략 적용 가능
- 단점
  - 문장이 중간에서 끊길 수 있음
  - 한 청크 = 한 의미가 잘 안 맞음
  - 문단/제목/섹션 등 문서 구조를 전혀 고려하지 않음

> **실무에서는 기본값 정도로만 사용하고,  
> 실제 서비스에서는 재귀/구조 기반 전략과 함께 튜닝하는 편이 좋다.**

##### ① 실습: PDF 로드 + 전체 텍스트 합치기

In [1]:
from langchain_community.document_loaders import PyPDFLoader

PDF_PATH = "data/농업기술.pdf"

loader = PyPDFLoader(PDF_PATH)
docs = loader.load()  # 페이지 단위 Document 리스트

print("페이지 수:", len(docs))
print("첫 페이지 내용 샘플:\n", docs[0].page_content[:500])

페이지 수: 16
첫 페이지 내용 샘플:
 2025. 11+12월호 
vol.695
농업기술
AGRICULTURAL TECHNOLOGY
간척지 구조물 안전시공
고성능 복합기둥 공법개발로 
다양한 지형에 폭넓게 적용
연두색 누에고치 품종
‘연녹잠’의 기능성과 가능성
혈당과 장건강까지 챙기는
‘도담쌀’
재배 안정성이 강화된
장원형 쌀 ‘케이롱’
축산 분야 
한기웅 명인
RDA 포커스
알쓸신농
추천! K품종
명인열전


- `docs`는 `Document` 객체 리스트이며, 각 객체는 `page_content`와 `metadata`를 가집니다.
- 이제 모든 페이지를 하나의 문자열로 합칩니다.

In [2]:
full_text = "\n".join([doc.page_content for doc in docs])
print("전체 텍스트 길이(문자 수):", len(full_text))

전체 텍스트 길이(문자 수): 14780


##### ② 실습: CharacterTextSplitter로 고정 길이 청크

`CharacterTextSplitter`는 지정한 기준(separator)으로 먼저 텍스트를 나눈 뒤,  
그 결과로 생성된 조각들을 설정한 chunk_size를 초과하지 않는 범위까지 차례대로 합쳐  
하나의 청크를 생성합니다.

In [3]:
from langchain_text_splitters import CharacterTextSplitter

fixed_splitter = CharacterTextSplitter(
    separator="\n",    # 줄바꿈을 기준으로 우선 쪼개고, (변경 가능)
    chunk_size=800,    # 최대 300자
    chunk_overlap=150,  # 앞 청크의 마지막 50자를 다음 청크와 겹쳐서 포함
    length_function=len
)

fixed_chunks = fixed_splitter.split_text(full_text)

print("고정 길이 청크 개수:", len(fixed_chunks))
print("첫 번째 청크 길이:", len(fixed_chunks[0]))
print("첫 번째 청크 내용 예시:\n", fixed_chunks[0])

고정 길이 청크 개수: 23
첫 번째 청크 길이: 764
첫 번째 청크 내용 예시:
 2025. 11+12월호 
vol.695
농업기술
AGRICULTURAL TECHNOLOGY
간척지 구조물 안전시공
고성능 복합기둥 공법개발로 
다양한 지형에 폭넓게 적용
연두색 누에고치 품종
‘연녹잠’의 기능성과 가능성
혈당과 장건강까지 챙기는
‘도담쌀’
재배 안정성이 강화된
장원형 쌀 ‘케이롱’
축산 분야 
한기웅 명인
RDA 포커스
알쓸신농
추천! K품종
명인열전
스마트폰으로 QR코드를 찍으면 ‘농업과학도서관’으로 연결되어 ‘농업기술’과 ‘농업기술길잡이’ 등
농촌진흥청에서 발행하는 다양한 책자를 만날 수 있습니다. 포털사이트에 ‘농업과학도서관’을 검색해보세요.
[54875] 전북특별자치도 전주시 덕진구 농생명로 300(중동)   T.1544-8572   F.063-238-1766   www.rda.go.kr
2026년, 한국 농업의 미래를 향한 
도전은 계속됩니다.
2025년, 
농촌진흥청이
만들어낸 성과
스마트 농업으로
현장 혁신기후 변화 대응 신품종과
탄소 절감 기술 개발
한국 농식품 수출 확대,
농식품 수출지원
치유농업 서비스로
삶을 풍요롭게
농업인 안전·재해 예방과
병해충 관리 기술 강화
RDA포커스
04	 	연두색	누에고치	품종	
‘연녹잠’의	기능성과	가능성
06	 	병에	강한	고추	종자를	찾아서	
08	 	간척지	구조물	안전	시공	위한	
고성능	복합기둥	공법	개발
10	 	국민	호흡기	건강	
잎들깨	‘숨들’로	지켜
12	 	에어로겔	다겹보온커튼으로,	
한파	걱정	끝!
14	 	효율성,	다양성	‘꽉’	
복숭아	디지털	육종	기술	개발


- `chunk_size`와 `chunk_overlap`은 나중에 성능 튜닝의 핵심 하이퍼파라미터가 됩니다.
- `separator="\n"` 덕분에 가능한 한 줄 단위로 쪼개고, 그래도 크면 300자 기준으로 자릅니다.
- 이 방식은 정확하진 않지만 아무 문서에나 바로 적용 가능하다는 장점이 있습니다.

#### 5.1.3 재귀적 청크 (Recursive Text Splitting)

`RecursiveCharacterTextSplitter`는 **큰 단위 → 작은 단위** 순으로 텍스트를 쪼개면서 `chunk_size`를 넘지 않도록 조절하는 전략입니다.

분할 단위는 다음과 같은 **우선순위(separators)** 로 처리됩니다:

1. **문단 단위**: `"\n\n"`
2. **문장/줄 단위**: `"\n"`
3. **단어 단위**: `" "`
4. **문자(char) 단위**: `""` (더 이상 나눌 수 없을 때 강제 분할)

**동작 원리(간단 버전)**

1. **가장 큰 separator(`\n\n`)로 먼저 텍스트를 분할한다.**  
2. 분리된 조각들 중 **chunk_size(예: 300)를 넘는 조각만** 다음 separator로 다시 분할한다.  
3. 다음 separator(`\n`)로 분할해도 크다면 → 단어 단위(`" "`)로 분할한다.  
4. 그래도 크다면 → 문자 단위로 강제 분할한다.  
5. 모든 조각이 chunk_size 이하가 될 때까지 위 과정을 반복한다.

➡ 결과적으로 텍스트는 **의미 단절을 최소화하면서도 chunk_size에 최대한 근접한 크기**로 분할됩니다.

##### ① 실습: 재귀 청크 생성

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150,
    separators=[
        "\n\n",  # 문단 단위
        "\n",    # 줄 단위
        ". ",    # 문장 단위 (간단한 예시)
        " ",     # 단어 단위
        ""       # 문자 단위 (최후의 수단)
    ]
)

recursive_chunks = recursive_splitter.split_text(full_text)

print("재귀적 청크 개수:", len(recursive_chunks))
print("첫 번째 재귀 청크 길이:", len(recursive_chunks[0]))
print("첫 번째 재귀 청크 예시:\n", recursive_chunks[0])

재귀적 청크 개수: 23
첫 번째 재귀 청크 길이: 764
첫 번째 재귀 청크 예시:
 2025. 11+12월호 
vol.695
농업기술
AGRICULTURAL TECHNOLOGY
간척지 구조물 안전시공
고성능 복합기둥 공법개발로 
다양한 지형에 폭넓게 적용
연두색 누에고치 품종
‘연녹잠’의 기능성과 가능성
혈당과 장건강까지 챙기는
‘도담쌀’
재배 안정성이 강화된
장원형 쌀 ‘케이롱’
축산 분야 
한기웅 명인
RDA 포커스
알쓸신농
추천! K품종
명인열전
스마트폰으로 QR코드를 찍으면 ‘농업과학도서관’으로 연결되어 ‘농업기술’과 ‘농업기술길잡이’ 등
농촌진흥청에서 발행하는 다양한 책자를 만날 수 있습니다. 포털사이트에 ‘농업과학도서관’을 검색해보세요.
[54875] 전북특별자치도 전주시 덕진구 농생명로 300(중동)   T.1544-8572   F.063-238-1766   www.rda.go.kr
2026년, 한국 농업의 미래를 향한 
도전은 계속됩니다.
2025년, 
농촌진흥청이
만들어낸 성과
스마트 농업으로
현장 혁신기후 변화 대응 신품종과
탄소 절감 기술 개발
한국 농식품 수출 확대,
농식품 수출지원
치유농업 서비스로
삶을 풍요롭게
농업인 안전·재해 예방과
병해충 관리 기술 강화
RDA포커스
04	 	연두색	누에고치	품종	
‘연녹잠’의	기능성과	가능성
06	 	병에	강한	고추	종자를	찾아서	
08	 	간척지	구조물	안전	시공	위한	
고성능	복합기둥	공법	개발
10	 	국민	호흡기	건강	
잎들깨	‘숨들’로	지켜
12	 	에어로겔	다겹보온커튼으로,	
한파	걱정	끝!
14	 	효율성,	다양성	‘꽉’	
복숭아	디지털	육종	기술	개발


##### ② Fixed vs Recursive 비교 포인트

실습 후 아래와 같이 비교해보자.

In [5]:
print("고정 청크 개수:", len(fixed_chunks))
print("재귀 청크 개수:", len(recursive_chunks))

print("\n[고정 청크 예시]")
print(fixed_chunks[0])

print("\n[재귀 청크 예시]")
print(recursive_chunks[0])

고정 청크 개수: 23
재귀 청크 개수: 23

[고정 청크 예시]
2025. 11+12월호 
vol.695
농업기술
AGRICULTURAL TECHNOLOGY
간척지 구조물 안전시공
고성능 복합기둥 공법개발로 
다양한 지형에 폭넓게 적용
연두색 누에고치 품종
‘연녹잠’의 기능성과 가능성
혈당과 장건강까지 챙기는
‘도담쌀’
재배 안정성이 강화된
장원형 쌀 ‘케이롱’
축산 분야 
한기웅 명인
RDA 포커스
알쓸신농
추천! K품종
명인열전
스마트폰으로 QR코드를 찍으면 ‘농업과학도서관’으로 연결되어 ‘농업기술’과 ‘농업기술길잡이’ 등
농촌진흥청에서 발행하는 다양한 책자를 만날 수 있습니다. 포털사이트에 ‘농업과학도서관’을 검색해보세요.
[54875] 전북특별자치도 전주시 덕진구 농생명로 300(중동)   T.1544-8572   F.063-238-1766   www.rda.go.kr
2026년, 한국 농업의 미래를 향한 
도전은 계속됩니다.
2025년, 
농촌진흥청이
만들어낸 성과
스마트 농업으로
현장 혁신기후 변화 대응 신품종과
탄소 절감 기술 개발
한국 농식품 수출 확대,
농식품 수출지원
치유농업 서비스로
삶을 풍요롭게
농업인 안전·재해 예방과
병해충 관리 기술 강화
RDA포커스
04	 	연두색	누에고치	품종	
‘연녹잠’의	기능성과	가능성
06	 	병에	강한	고추	종자를	찾아서	
08	 	간척지	구조물	안전	시공	위한	
고성능	복합기둥	공법	개발
10	 	국민	호흡기	건강	
잎들깨	‘숨들’로	지켜
12	 	에어로겔	다겹보온커튼으로,	
한파	걱정	끝!
14	 	효율성,	다양성	‘꽉’	
복숭아	디지털	육종	기술	개발

[재귀 청크 예시]
2025. 11+12월호 
vol.695
농업기술
AGRICULTURAL TECHNOLOGY
간척지 구조물 안전시공
고성능 복합기둥 공법개발로 
다양한 지형에 폭넓게 적용
연두색 누에고치 품종
‘연녹잠’의 기능성과 가능성
혈당과 장건강까지 챙기는
‘도담쌀’
재배 안정성이 강화된
장원형 쌀 ‘케이롱’
축산 분야 
한기웅 명인
RDA

비교해볼 때 관찰 포인트:

- 문단이 중간에서 끊기는지 여부  
- 한 청크 안에 주제가 여러 개 섞여 있는지  
- 첫 문장/마지막 문장이 자연스럽게 끝나는지  

#### 5.1.4 문서 구조 기반 청크 (Structure-aware Chunking)

보고서·매뉴얼·가이드라인처럼 **문서 구조(Heading, 표제어, 섹션)** 가 중요할 때 매우 효과적.  
제 1장, 제 2장 등 특정 형식으로 되어있을 경우 이 형식을 이용하여 효과적으로 분할 가능

- **방식:**  
  - H1/H2/H3, Section Title 등으로 먼저 분할  
  - 이후 필요한 길이만큼 세부 조정

- **장점:**  
  - 검색 시 특정 "섹션 전체"가 안정적으로 반환  
  - 기업/공공기관 문서에 특히 강력  
  - "근거 제시" 품질이 매우 높아짐

- **적합한 경우:**  
  - 규정집, 공장 매뉴얼, 기술 문서, SOP, 제안서, 법령 등

##### ② 실습: 매우 단순한 헤더 기반 분리 예시

In [6]:
import re

def split_by_headings(text: str):
    # 예시 패턴: "제1장", "1.", "2.", "3." 등으로 시작하는 라인
    # 실제 환경에서는 도메인에 맞게 정규식을 더 정교하게 다듬어야 함
    pattern = r"(?:^|\n)(제\s*\d+장.+|\d+\.\s.+)"
    splits = re.split(pattern, text)

    sections = []
    for i in range(0, len(splits), 2):
        body = splits[i].strip()
        if i + 1 < len(splits):
            header = splits[i + 1].strip()
            section_text = f"{header}\n{body}"
        else:
            section_text = body

        if section_text.strip():
            sections.append(section_text.strip())

    return sections

structured_sections = split_by_headings(full_text)

print("헤더 기반 1차 분할 개수:", len(structured_sections))
print("첫 섹션 예시:\n", structured_sections[0][:500])

헤더 기반 1차 분할 개수: 8
첫 섹션 예시:
 2025. 11+12월호


##### ③ 각 섹션에 재귀 청크 적용

In [7]:
structured_chunks = []
for sec in structured_sections:
    chunks = recursive_splitter.split_text(sec)
    structured_chunks.extend(chunks)

print("구조 기반 최종 청크 개수:", len(structured_chunks))
print("구조 기반 첫 청크 예시:\n", structured_chunks[0])

구조 기반 최종 청크 개수: 26
구조 기반 첫 청크 예시:
 2025. 11+12월호


이 방식의 장점:

- 같은 섹션 안에서만 잘린 청크라 문맥이 훨씬 안정적  
- 검색 시, 특정 섹션과 관련된 질의가 왔을 때 **관련 내용이 묶여서 잘 검색**됨  
- 공공문서 / 매뉴얼 / 규정집에서 특히 강력한 효과

#### 5.1.5 정리

이 절에서 한 것:

1. `CharacterTextSplitter`로 **고정 길이 청크** 만들어 보기  
2. `RecursiveCharacterTextSplitter`로 **의미 보존 + 길이 제약을 동시에 고려한 청크** 만들기  
3. 간단한 정규식을 이용해 **문서 구조를 의식한 청크** 만들어 보기  

이제 이 청크들을 임베딩해서 벡터DB에 저장하면,  
**같은 문서라도 어떤 청킹 전략을 쓰느냐에 따라 검색 결과가 달라집니다.**

다음 절에서는 동일한 임베딩/벡터DB를 사용하되  
- **쿼리(Query)를 어떻게 변형(Transformation)하면 검색 품질이 올라가는지**  
- Query Rewriting, Multi-Query, HyDE 등의 기법을 다룹니다.


##### 5.1.6 벡터 DB 생성

이후 5.2절부터 진행될 RAG 실습에서 사용할 **벡터 DB**를 생성합니다.  
이 단계에서는 이전 실습에서 이미 생성해 둔 벡터 DB가 있는지 먼저 확인하고,  
존재할 경우 이를 그대로 불러와 재사용합니다.  

만약 저장된 벡터 DB가 없다면,  
PDF 문서를 로드하고 청크 분할 및 임베딩 과정을 거쳐  
새로운 벡터 DB를 생성한 뒤 로컬에 저장합니다.  

이를 통해 불필요한 재계산을 줄이고,  
이후 실습에서는 동일한 벡터 DB를 기반으로 검색 및 생성 과정을 진행할 수 있습니다.

In [8]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()

# 1. 문서 로드 & 청크 분할
PDF_PATH = "data/농업기술.pdf"  
DB_PATH = "./my_faiss_index"

# 임베딩 모델 준비
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 2. 로컬 DB 존재 여부 확인 및 분기 처리
if os.path.exists(DB_PATH):
    # (A) 이미 저장된 DB가 있는 경우 -> 불러오기
    print(f"기존 벡터 DB를 '{DB_PATH}'에서 불러옵니다...")
    
    vectorstore = FAISS.load_local(
        folder_path=DB_PATH, 
        embeddings=embeddings, 
        allow_dangerous_deserialization=True # 필수 설정
    )
    print("DB 로드 완료!")

else:
    # (B) 저장된 DB가 없는 경우 -> 새로 생성 및 저장
    print("저장된 DB가 없습니다. 문서를 로드하고 새로 생성합니다...")

    # --- 문서 로드 & 청크 분할 (기존 코드) ---
    loader = PyPDFLoader(PDF_PATH)
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=150,
        separators=["\n\n", "\n", " ", ""],
    )

    split_docs = text_splitter.split_documents(docs)
    
    # 메타데이터 ID 부여
    for idx, d in enumerate(split_docs):
        d.metadata["id"] = idx

    print(f"원본 페이지 수: {len(docs)}")
    print(f"분할된 청크 수: {len(split_docs)}")
    
    # --- 벡터 DB 생성 및 저장 ---
    vectorstore = FAISS.from_documents(split_docs, embeddings)
    
    # 로컬에 저장
    vectorstore.save_local(DB_PATH)
    print(f"벡터 DB가 '{DB_PATH}' 경로에 저장되었습니다.")

# Retriever 생성 (RAG에서 사용할 검색기)
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 4}  # 관련도 높은 청크 4개 정도 가져오기
)

기존 벡터 DB를 './my_faiss_index'에서 불러옵니다...
DB 로드 완료!


### 5.2 Pre-Retrieval 최적화
**검색 품질의 절반은 사용자 쿼리를 어떻게 변형하느냐로 결정됩니다.**

Chunking이 문서를 최적으로 쪼개는 기술이라면,  
**Query Transformation은 검색을 최적화하기 위해 사용자의 질문을 개선하는 기술**입니다.

실제 RAG 성능 문제 중 상당수가  
- 쿼리가 모호하거나  
- 문법적으로 어색하거나  
- 정보가 부족하거나  
- 다양한 표현을 제대로 포착하지 못해서  
발생한다.

따라서 Query Transformation은 Advanced RAG에서 매우 중요한 요소입니다.

본 절에서는 다음 기법들을 다룹니다.

1. Query Rewriting (LLM 기반)
2. Query Expansion
3. Multi-Query Retrieval
4. HyDE (Hypothetical Document Embedding)

#### 5.2.1 Query Transformation이 필요한 이유

##### ① 사용자의 질문은 종종 “원본 문서 표현”과 정확히 일치하지 않음  
예:  
- 질문: *“농업 기술 보급 기준 알려줘”*  
- 문서: *“농업 기술의 확산을 위한 지원 지침…”*

문서는 “보급”이 아니라 “확산”,  
“기준”이 아니라 “지침”이라는 표현을 쓰고 있음.

→ **표면적으로 단어가 다르면 벡터 검색 Recall이 크게 떨어짐**

##### ② 사용자의 질문은 종종 맥락이 부족함  
예:  
“그 정책 기준 뭐였지?”  
→ 무엇을 의미하는지 검색 입장에서 알 수 없음.

##### ③ LLM은 “자유로운 언어 생성” 능력은 뛰어나지만  
     “검색 가능한 형태로 쿼리를 바꿔주는 능력”은 별도 설계가 필요함  
즉, **검색 친화적(queryable) 문장으로 바꾸는 과정이 필요**하다.

#### 5.2.2 Query Rewriting (LLM 기반)

사용자의 질문을 LLM을 이용해 “검색 친화적인 문장”으로 다시 작성하는 방식입니다.

예:

사용자 입력:
> “농업 정책 바뀐 기준 알려줘”

LLM이 rewriting:
> “최근 개정된 농업 기술 보급 및 지원 정책 기준에 관한 문서 정보를 검색하라.”

##### 장점
- 질문의 모호함 제거  
- 불완전한 문장을 온전한 질의 형태로 개선  
- 도메인 특성 키워드 주입 가능

##### Query Rewriting 실습

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

rewrite_prompt = PromptTemplate.from_template("""
다음 사용자의 질문을 검색 최적화된 형태로 다시 작성하세요.
- 의미는 유지하되 더 명확하고 완전한 문장으로 바꿉니다.
- 핵심 키워드를 추가하거나 정제해도 좋습니다.

질문: {query}
""")

def rewrite_query(query):
    rewritten = llm.invoke(rewrite_prompt.format(query=query))
    return rewritten.content

query = "농업 테크놀로지 네임 뭐임"
new_query = rewrite_query(query)
print("Rewritten Query:", new_query)

Rewritten Query: 농업 기술의 이름은 무엇인가요?


In [10]:
retriever.invoke(query)

[Document(id='021fa4c8-0585-416d-aff5-498806c4e6fa', metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20251120203111', 'source': 'data/농업기술.pdf', 'total_pages': 16, 'page': 4, 'page_label': '5', 'id': 4}, page_content='글 이지혜 농업연구사    농촌진흥청 국립농업과학원 산업곤충과 063-238-294401\nRDA \n포커스\n누에를 키우는 농업인 양잠(養蠶)은 산업과 함께 변화했다. 과거 우리나라의 양잠은 섬유산업으로서 전성기를 누렸\n다. 실크를 만들기 위한 누에고치는 하얗고, 흠이 적으며, 두께가 균일하고, 질긴 특성을 가지는 것이 상품(上品)이었\n다. 시대가 흘러 양잠은 기능성 산업으로 전환기를 맞는다. 누에 분말은 혈당 강하 소재로, 누에고치 추출물은 기억\n력 개선 소재로 가치를 찾고, 건강기능식품으로 등록되는 성과를 거두었다. 그에 따라 양잠의 기준도 달라졌다. 기능\n성 높은 시기에 수확하고, 기능성 높은 품종이 선호되었다. 이에 농촌진흥청에서는 기능성 양잠산업 육성을 위한 기\n능성 품종 개발에 힘쓰고 있다. 그중에서도 연두색 누에고치를 짓는‘연녹잠’의 매력을 소개한다.\n연두색 고치 품종, ‘연녹잠’\n누에는 뽕잎을 먹고 자란다. 누에 품종에 따른 실샘(실크를 만들어내는 기관) 차이에 따라 누에고치에는 뽕잎 성\n분이 전달되지만 그렇지 못한 경우도 있다. 뽕잎 성분이 전달되지 못한 고치는 백색, 카로티노이드가 전달된 고치\n는 황색, 플라보노이드가 전달된 고치는 녹색으로 나타난다. 농촌진흥청에서 2009년에 개발한 연녹잠은 이름에\n서도 알 수 있듯이 플라보노이드가 축적된 연두색 고치를 만드는 품종이다. 연녹잠 고치에는 뽕잎에서 유래한 플\n라보노이드에 당이 결합한 새로운 구조로 축적되어 있으며, 생육 특성이 우수하여 키우기 쉽다

In [11]:
retriever.invoke(new_query)

[Document(id='1d619bdd-a3dc-49d7-b4b3-cd58ea5b1cc4', metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20251120203111', 'source': 'data/농업기술.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1', 'id': 0}, page_content='2025. 11+12월호 \nvol.695\n농업기술\nAGRICULTURAL TECHNOLOGY\n간척지 구조물 안전시공\n고성능 복합기둥 공법개발로 \n다양한 지형에 폭넓게 적용\n연두색 누에고치 품종\n‘연녹잠’의 기능성과 가능성\n혈당과 장건강까지 챙기는\n‘도담쌀’\n재배 안정성이 강화된\n장원형 쌀 ‘케이롱’\n축산 분야 \n한기웅 명인\nRDA 포커스\n알쓸신농\n추천! K품종\n명인열전'),
 Document(id='6e334fa1-631c-4e70-a34d-d8783fbcfe8c', metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20251120203111', 'source': 'data/농업기술.pdf', 'total_pages': 16, 'page': 11, 'page_label': '12', 'id': 20}, page_content='적인 공급망을 마련하고, 이를 통해 농가 소득 \n증대와 국산 기능성 원료 산업의 성장을 동시\n에 추진하고 있다. \n식물특허 보호 중으로 원료 사용은 \n기술이전 절차 거쳐야\n특히 중요한 점은 ‘숨들’이 식물특허로 보호되\n고 있다는 사실이다. 따라서 기업에서 원료로 \n활용하려면 반드시 한국농업기술진흥원을 통\n한 기술이전 절차를 거쳐야 하며, 정식 권리 \n이전이 이루어져야 종자를 공급받아 사용할 \n수 있다. \n앞으로는 임상 연구를 통해 ‘숨들’의 기능성을 \n더욱 심화시키고, 건강기능식품뿐

위 결과에서 볼 수 있듯이, **질문의 표현이 달라지면 검색 결과 역시 달라집니다.**

검색 단계에서 유의미한 청크를 얻지 못했다면,  
부정확한 답변을 생성하기 전에 **질문을 바꿔 다시 검색해보는 것이 중요합니다.**

이때 Query Rewrite는 필수 기능이라기보다는,  
**검색 품질을 보완하고 RAG 성능을 안정화하기 위한 중요한 보조 기능**으로 활용될 수 있습니다.


#### 5.2.3 Query Expansion (Semantic Query Paraphrasing)

Dense Retrieval에서는 **질문을 한 가지 표현으로만 검색할 경우** 문서에 존재하는 다양한 서술 방식·관점을 충분히 포착하지 못해  
**Recall 손실**이 발생할 수 있습니다.

이를 보완하기 위해 최신 RAG에서는 **키워드 확장보다는, 질문 자체를 여러 의미적 관점에서 다시 쓰는 패러프레이징 기법**을 사용합니다.

##### 핵심 개념
- Query Expansion = 원 질문을 **의미적으로 재구성하는 여러 형태의 질문 세트 생성**
- 서로 다른 맥락·관점을 가진 질문을 생성하여 Retriever가 놓친 문서를 회수
- Dense Retriever에 특히 효과적이며, Sparse BM25의 키워드 확장과는 목적이 다름

##### 예시

원본 쿼리: 농업 기술 지원 정책의 효과 분석

패러프레이징된 확장 쿼리 예:

- 농업 기술 보급 정책이 실제 농가에 미친 성과 평가  
- 정부의 농업 기술지원 프로그램 효과 분석  
- 기술 지원 제도의 결과와 영향을 다룬 연구  
- 농업 기술 확산 정책의 실질적 성과 평가  
- 농가 지원 정책이 기술 도입에 미친 영향 분석  

이처럼 **서로 다른 표현 방식의 질문을 병렬로 검색**한 뒤 결과를 합쳐 reranking하면 검색 성능이 크게 향상됩니다.

##### Query Expansion 실습

In [12]:
paraphrase_prompt = PromptTemplate.from_template("""
아래 질문과 의미적으로 같은 내용을 서로 다른 관점으로 표현한 
패러프레이즈 문장을 5개 생성하세요.

조건:
- 질문의 의미는 유지
- 표현만 다르게 작성
- 줄바꿈(\n)으로만 구분된 리스트 형식으로 출력할 것

질문: {query}
""")

def expand_query_paraphrase(query):
    resp = llm.invoke(paraphrase_prompt.format(query=query)).content
    # 응답을 줄 단위 리스트로 변환
    return [line.strip() for line in resp.split("\n") if line.strip()]

# 테스트
expanded_queries = expand_query_paraphrase("농업 기술 지원 정책의 효과 분석")
print(expanded_queries)

['- 농업 기술 지원 정책이 미치는 영향에 대한 분석', '- 농업 기술 지원 정책의 효과에 대한 평가', '- 농업 기술 지원 정책이 가져오는 결과를 분석하기', '- 농업 기술 지원 정책의 성과를 검토하는 연구', '- 농업 기술 지원 정책의 효율성을 조사하는 작업']


**활용**:  
- 확장 키워드를 기반으로 multi-vector 검색  
- 또는 BM25와 함께 hybrid 검색 구성

#### 5.2.4 Multi-Query Retrieval (다중 질문 검색)

**Multi-Query Retrieval**은 앞선 **5.2.3 Query Expansion(패러프레이징)** 을  
**Retrieval 단계에 자동으로 통합한 구조적 기법**이다.

여기서는 이 전략을 **실제 검색 시스템에서 자동으로 실행하는 구현 단계**를 다룹니다.

즉, Multi-Query Retrieval은 다음 세 단계를 자동화한 패키지 구조다.

1. **LLM이 원 질문을 여러 관점으로 패러프레이징 생성**
2. **각 쿼리를 Retriever에 독립적으로 질의**
3. **모든 결과를 병합(Fusion)하여 상위 k 문서 추출**

이 방식은 dense retriever가 놓친 문서를 다른 질문 표현이 잡아주는 구조이므로  
**Recall이 크게 증가**하고 RAG 응답 품질의 안정성을 높인다.

##### 예시 흐름

원본 질문:
> “농업 기술 지원 기준 알려줘”

LLM이 자동 생성한 확장 쿼리 예:
- “농업 기술 보급 정책 기준은 무엇인가?”
- “최근 개정된 농업 기술 지원 지침 요약”
- “농업 기술 관련 정부 규정 정리”

각 질문을 검색 → 결과를 모두 합침 →  RRF(Fusion Scoring)로 중복 제거 + 점수 조합 → 최종 k개 문서 반환.


##### 실습

In [13]:
from langchain_classic.retrievers.multi_query import MultiQueryRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS


# LLM 설정
llm = ChatOpenAI(
    model="gpt-4o-mini",   # 필요 시 다른 모델로 교체 가능
    temperature=0.1,       # 최대한 사실 위주의 응답을 위해 낮게 설정
)

# Multi-Query Retrieval 구성
# 내부적으로 LLM이 질문을 paraphrase하여 여러 개 생성함
multi_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever,
    llm=llm,
)

# 테스트
query = "연녹잠에 대해서 알려줘"
results = multi_retriever.invoke(query)

print("검색된 문서 수:", len(results))
for r in results[:3]:
    print("---")
    print(r.page_content[:300])

검색된 문서 수: 6
---
하여 연녹잠 고치로 스마트섬유를 만들어 볼 
수 있을 것이다. 피부미용에도 효과가 좋았다. 
연녹잠 홍잠(고치 짓기 전 성숙한 누에를 찐 
것)을 섭취한 실험쥐에서 멜라닌 생성이 억제
되고, 피부밝기가 41% 개선되었다. 이와 같
이 연녹잠은 기능성 누에 품종으로 식의약 소
재 외에도 화장품, 바이오소재 등 다양한 분야
에 활용될 수 있을 것으로 보인다.
‘연녹잠’, 우수 누에 품종의 보급
다양한 잠재력을 가진 연녹잠은 우수 누에 품
종으로 지정되어 농촌진흥청에서 매년 누에씨
를 지자체 유관기관과 협력하여 보급하고 있
다. 농가수요
---
연녹잠 추출물연녹잠 추출물
콜레스테롤 콜레스테롤 
효과적으로 감소효과적으로 감소
‘연녹잠’누에고치 추출물의 
콜레스테롤 조절 효과
연녹잠 누에고치 추출물은 콜레스테롤 대사 
개선에 효과를 보였다. 고지방식이로 비만을 
유도한 마우스 모델에 투여하였을 때, 나쁜 콜
레스테롤로 알려진 저밀도 지단백(LDL) 콜레
스테롤의 혈중농도가 26% 감소했다. 이는 콜
레스테롤 치료제로 알려진 스타틴(Statin)과 
유사한 수준이다. 간 조직의 콜레스테롤 함량
도 분석해 보았다. 연녹잠은 스타틴과 달리 간
에서 콜레스테롤을 배출하는 기전을 활성
---
글 이지혜 농업연구사    농촌진흥청 국립농업과학원 산업곤충과 063-238-294401
RDA 
포커스
누에를 키우는 농업인 양잠(養蠶)은 산업과 함께 변화했다. 과거 우리나라의 양잠은 섬유산업으로서 전성기를 누렸
다. 실크를 만들기 위한 누에고치는 하얗고, 흠이 적으며, 두께가 균일하고, 질긴 특성을 가지는 것이 상품(上品)이었
다. 시대가 흘러 양잠은 기능성 산업으로 전환기를 맞는다. 누에 분말은 혈당 강하 소재로, 누에고치 추출물은 기억
력 개선 소재로 가치를 찾고, 건강기능식품으로 등록되는 성과를 거두었다. 그에 따라 양잠


지금까지는 쿼리를 변형해 검색 품질을 높이는 방법을 봤습니다.  
이제는 검색기 자체를 설계/조합하는 HyDE와 Hybrid Retrieval을 살펴봅니다

#### 5.2.5 HyDE (Hypothetical Document Embedding)

HyDE는 LLM이 질문에 대한 **가상의 답변 문서**를 생성하고, 이 문서를 임베딩하여 검색 쿼리로 사용하는 방식입니다. 

이 접근은 질문 문장이 짧고 질문형 구조를 띠어 실제 정책·기술 문서와 표현 방식이 다를 수 있다는 점에서 출발합니다.  

항상 그런 것은 아니지만, 여러 벤치마크에서  
- 질문은 짧고 질문형 구조를 띠고,  
- 실제 문서는 서술형·설명형 구조를 띠기 때문에  

질문 임베딩보다 **답변 형태의 가상 문서 임베딩이 실제 정답 문서 근처에 위치하는 사례가 자주 관찰**되었습니다.  
이로 인해 전체적으로 Recall 향상 효과가 보고되고 있습니다.

다만 이는 문서 유형, 질문 형태, 도메인 특성에 따라 달라질 수 있으며, 언제나 더 좋은 벡터를 제공한다고 단정하기는 어렵습니다.

##### 예시 질문  
> “농업 기술 보급 기준은?”

LLM이 생성한 가상 문서  
> “농업 기술 보급 기준은 정부의 기술 확산 지침에 따라 …”

##### HyDE 실습

In [14]:
hyde_prompt = PromptTemplate.from_template("""
아래 질문에 대한 '가상의 답변 문서'를 작성해 주세요.
이 문서는 실제 지식이 아니라 검색을 돕기 위한 요약/설명 형식이어야 합니다.
질문: {query}
""")
query = "연녹잠 누에고치 추출물의 효과를 알려줘"
hypo = llm.invoke(hyde_prompt.format(query=query)).content
print(f'가상의 답변 문서 : \n {hypo}')
results = vectorstore.similarity_search(hypo, k=5)

가상의 답변 문서 : 
 # 연녹잠 누에고치 추출물의 효과에 대한 가상의 답변 문서

## 개요
연녹잠 누에고치 추출물은 전통적으로 사용되어 온 자연 유래 성분으로, 최근에는 다양한 연구를 통해 그 효능이 주목받고 있습니다. 이 문서에서는 연녹잠 누에고치 추출물의 주요 효과와 활용 가능성에 대해 설명합니다.

## 주요 효과

### 1. 항산화 작용
연녹잠 누에고치 추출물은 강력한 항산화 성분을 함유하고 있어, 체내의 자유 라디칼을 제거하는 데 도움을 줄 수 있습니다. 이는 노화 방지 및 세포 손상을 예방하는 데 기여할 수 있습니다.

### 2. 피부 개선
연녹잠 누에고치 추출물은 피부 보습 및 탄력 증진에 효과적이라고 알려져 있습니다. 이 성분은 피부 장벽을 강화하고, 수분 손실을 줄여주어 건강하고 빛나는 피부를 유지하는 데 도움을 줄 수 있습니다.

### 3. 항염 효과
연구에 따르면, 연녹잠 누에고치 추출물은 염증을 줄이는 데 도움을 줄 수 있는 성분을 포함하고 있습니다. 이는 여드름, 아토피 피부염 등 다양한 피부 질환의 개선에 기여할 수 있습니다.

### 4. 면역력 증진
연녹잠 누에고치 추출물은 면역 체계를 강화하는 데 도움을 줄 수 있는 성분을 포함하고 있어, 감염 예방 및 전반적인 건강 증진에 기여할 수 있습니다.

### 5. 항균 효과
일부 연구에서는 연녹잠 누에고치 추출물이 특정 세균에 대한 항균 효과를 나타낸다고 보고되었습니다. 이는 피부 감염 예방 및 치료에 유용할 수 있습니다.

## 활용 방법
연녹잠 누에고치 추출물은 다양한 형태로 활용될 수 있습니다. 주로 화장품, 스킨케어 제품, 건강 보조 식품 등에서 사용되며, 피부에 직접 적용하거나 섭취하는 방식으로 효과를 기대할 수 있습니다.

## 결론
연녹잠 누에고치 추출물은 항산화, 피부 개선, 항염, 면역력 증진, 항균 효과 등 다양한 효능을 가진 성분으로, 현대의학과 전통의학에서 모두 주목받고 있습니다. 그러나 개인의 체질이나 피부 상태에 따라 효과가 다를 수 있으므로, 사용

In [15]:
for r in results:
    print("--- 결과 ---")
    print(r.page_content[:200])

--- 결과 ---
연녹잠 추출물연녹잠 추출물
콜레스테롤 콜레스테롤 
효과적으로 감소효과적으로 감소
‘연녹잠’누에고치 추출물의 
콜레스테롤 조절 효과
연녹잠 누에고치 추출물은 콜레스테롤 대사 
개선에 효과를 보였다. 고지방식이로 비만을 
유도한 마우스 모델에 투여하였을 때, 나쁜 콜
레스테롤로 알려진 저밀도 지단백(LDL) 콜레
스테롤의 혈중농도가 26% 감소했다. 이는 콜

--- 결과 ---
하여 연녹잠 고치로 스마트섬유를 만들어 볼 
수 있을 것이다. 피부미용에도 효과가 좋았다. 
연녹잠 홍잠(고치 짓기 전 성숙한 누에를 찐 
것)을 섭취한 실험쥐에서 멜라닌 생성이 억제
되고, 피부밝기가 41% 개선되었다. 이와 같
이 연녹잠은 기능성 누에 품종으로 식의약 소
재 외에도 화장품, 바이오소재 등 다양한 분야
에 활용될 수 있을 것으로 보인다.

--- 결과 ---
글 이지혜 농업연구사    농촌진흥청 국립농업과학원 산업곤충과 063-238-294401
RDA 
포커스
누에를 키우는 농업인 양잠(養蠶)은 산업과 함께 변화했다. 과거 우리나라의 양잠은 섬유산업으로서 전성기를 누렸
다. 실크를 만들기 위한 누에고치는 하얗고, 흠이 적으며, 두께가 균일하고, 질긴 특성을 가지는 것이 상품(上品)이었
다. 시대가 흘러 양잠은
--- 결과 ---
글 노나영 농업연구사    농촌진흥청 국립농업과학원 농업유전자원센터 063-238-485102
RDA 
포커스
고추는 김치, 고추장 등 우리 식탁에서 빠질 수 없는 대표적인 채소이다. 이렇게 소중한 고추가 해마다 풋마름
병, 각종 바이러스 병으로 인해 큰 피해를 입고 있다. 특히 장마철이 되면 탄저병 같은 병해가 심하게 발생해 수
확량이 크게 줄어들고, 농
--- 결과 ---
적인 공급망을 마련하고, 이를 통해 농가 소득 
증대와 국산 기능성 원료 산업의 성장을 동시
에 추진하고 있다. 
식물특허 보호 중으로 원료 사용은 
기술이전 절차 거쳐야
특히 중요한 점은 ‘숨들’이 식물특허로 보호되
고 있다는 사실이다. 따라서 기업에

#### 5.2.6 Hybrid Retrieval

**Hybrid Retrieval**은 Advanced RAG의 핵심 기법 중 하나입니다.  

왜냐하면 **단일 검색 방식(dense-only 또는 keyword-only)** 으로는 현실 세계의 문서를 안정적으로 검색하기 어렵기 때문입니다.



| 방식 | 장점 | 단점 |
|------|------|------|
| 벡터 기반 의미 검색 | 표현이 달라도 의미가 같으면 잘 잡아냄 | ~법 제~조와 같은 키워드 법령·표기 검색에 약함 |
| 키워드 기반 매칭 검색| 키워드 검색에 매우 강함. 법령구문·코드·표기 정확도 높음   | 표현이 조금만 달라도 검색 실패|
| Hybrid | 둘 다 잡아서 가장 높은 Recall | 시스템 복잡성 증가 | 

이 둘은 서로 **보완적인 강점**을 가지기 때문에, Hybrid Retrieval은 두 방식의 장점을 결합해  
**Recall(검색 누락 방지)과 Precision(정확성) 모두를 향상**시키는 전략입니다.

##### Hybrid Retrieval의 구성 방식

Hybrid Retrieval에는 몇 가지 대표적인 방식이 있습니다.

| 방식 | 설명 |
|------|------|
| **Parallel Retrieval** | Dense 검색 + BM25 검색 결과를 모두 가져온 후 합침 |
| **Weighted Score Fusion** | 두 검색 방식의 점수를 정규화 후 가중 평균 |
| **Reciprocal Rank Fusion (RRF)** | 서로 다른 검색 결과를 순위 기반으로 자연스럽게 합침 |
| **EnsembleRetriever** | LangChain에서 제공하는 하이브리드 조합기 |

이 중 **가장 많이 쓰이는 방식은 RRF**이며 해당 방식으로 실습을 진행해 보겠습니다.

##### Hybrid Retrieval 실습

In [16]:
from langchain_community.retrievers import BM25Retriever
# 1. 문서 로드 & 청크 분할
PDF_PATH = "data/농업기술.pdf"  

# --- 문서 로드 & 청크 분할 (기존 코드) ---
loader = PyPDFLoader(PDF_PATH)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150,
    separators=["\n\n", "\n", " ", ""],
)

split_docs = text_splitter.split_documents(docs)

# 메타데이터 ID 부여
for idx, d in enumerate(split_docs):
    d.metadata["id"] = idx
    
# (3) BM25 sparse retriever 구성
bm25_retriever = BM25Retriever.from_documents(split_docs)
dense_retriever = retriever

#### 5.2.6.1 Parallel Hybrid Retrieval (기본 조합)

Dense 검색과 Sparse 검색을 병렬로 수행한 후 결과를 단순 합칩니다.

In [17]:
query = "농업 기술 지원 기준 알려줘"

dense_docs = dense_retriever.invoke(query)
sparse_docs = bm25_retriever.invoke(query)

print("Dense 결과 수:", len(dense_docs))
print("Sparse 결과 수:", len(sparse_docs))

# 단순 합치기
combined = dense_docs + sparse_docs
print("Combined 결과 수:", len(combined))

Dense 결과 수: 4
Sparse 결과 수: 4
Combined 결과 수: 8


In [18]:
print("dense 결과:")
for i in dense_docs:
    print(i.page_content[:10])
print("------------")
print("sparse 결과:")
for i in sparse_docs:
    print(i.page_content[:10])

dense 결과:
도 폭넓게 적용할 
2025. 11+1
글 이학성 농업연구
알쓸신농
22	 	
------------
sparse 결과:
스마트폰으로 QR코
근법으로 전환되는 
도 폭넓게 적용할 
RDA포커스
04	


→ 단순하지만, 중복이 많고 순위가 섞여 있어 중복 제거 및 정렬이 필요함.

#### 5.2.6.2 Reciprocal Rank Fusion (RRF)

**(검색 단계 Fusion)** 서로 다른 검색 전략의 결과를 합쳐 Recall을 높이는 기법

RRF 공식:
```
score = 1 / (k + rank)
```
- rank = 검색 순위(1위면 1, 10위면 10)  
- k = 보통 60 고정  
→ 순위가 높을수록 큰 점수  
→ 두 방식에서 모두 높은 순위를 받을수록 최종 점수 증가

LangChain에서는 Auto-RRF 방식이 이미 구현되어 있습니다.


In [19]:
def reciprocal_rank_fusion(result_lists, k=60):
    scores = {}

    for results in result_lists:
        for rank, doc in enumerate(results):
            # RRF 공식 적용: 1 / (k + rank)
            score = 1.0 / (k + rank + 1)

            # Document ID 기준으로 점수 누적
            doc_id = doc.metadata.get("id", doc.page_content[:50])

            if doc_id not in scores:
                scores[doc_id] = {"doc": doc, "score": 0}
            scores[doc_id]["score"] += score

    # 점수 기준 정렬
    fused = sorted(scores.values(), key=lambda x: x["score"], reverse=True)
    return [x["doc"] for x in fused]

def hybrid_rrf_retrieve(query, retrievers, k=60):
    # 각 retriever로 검색
    results = [r.invoke(query) for r in retrievers]

    # RRF 점수로 병합
    fused_results = reciprocal_rank_fusion(results, k=k)
    return fused_results

dense_results = dense_retriever.invoke(query)
bm25_results = bm25_retriever.invoke(query)

hybrid_results = hybrid_rrf_retrieve(
    query,
    retrievers=[dense_retriever, bm25_retriever],
    k=60
)

print("RRF Hybrid 결과 수:", len(hybrid_results))

for r in hybrid_results[:3]:
    print("---")
    print(r.page_content[:300])

RRF Hybrid 결과 수: 7
---
도 폭넓게 적용할 수 있다. 향후 지능형 농장
인 스마트팜, 축사, 농촌 주택 등 경량 구조물
의 기초기술로도 확장하여 적용할 수 있다. 이 
기술은 2025년 5월 특허등록하고, 국제 학술
지 카본(Carbon, IF 10.5) 등에 총 4편의 관련 
SCI(E) 논문을 게재했다. 
 복합기둥 공법의 온실시공비
구분 나무 기둥 콘크리트 기둥 나무-FRP 복합기둥 개선 사항
인발 저항력 22.4kN - 33.2kN ▲ 48.2% 향상
내부식성 부식 위험 존재 20년 이상 유지 20년 이상 유지 안정성 확보
시공공정 4단계(기초·기둥
---
스마트폰으로 QR코드를 찍으면 ‘농업과학도서관’으로 연결되어 ‘농업기술’과 ‘농업기술길잡이’ 등
농촌진흥청에서 발행하는 다양한 책자를 만날 수 있습니다. 포털사이트에 ‘농업과학도서관’을 검색해보세요.
[54875] 전북특별자치도 전주시 덕진구 농생명로 300(중동)   T.1544-8572   F.063-238-1766   www.rda.go.kr
2026년, 한국 농업의 미래를 향한 
도전은 계속됩니다.
2025년, 
농촌진흥청이
만들어낸 성과
스마트 농업으로
현장 혁신기후 변화 대응 신품종과
탄소 절감 기술 개발
한국 농식품 수
---
2025. 11+12월호 
vol.695
농업기술
AGRICULTURAL TECHNOLOGY
간척지 구조물 안전시공
고성능 복합기둥 공법개발로 
다양한 지형에 폭넓게 적용
연두색 누에고치 품종
‘연녹잠’의 기능성과 가능성
혈당과 장건강까지 챙기는
‘도담쌀’
재배 안정성이 강화된
장원형 쌀 ‘케이롱’
축산 분야 
한기웅 명인
RDA 포커스
알쓸신농
추천! K품종
명인열전


##### RRF의 장점
- 서로 다른 검색 전략의 결과를 **균형 있게 합침**  
- 단순 가중 평균보다 훨씬 안정적  
- 실제 Enterprise RAG에서도 가장 선호되는 방식

#### 5.2.6.3 Hybrid Retrieval 결과 비교 실험

다음 코드를 사용하면 Dense, Sparse, Hybrid(RRF) 결과를 비교할 수 있습니다.

In [20]:
def preview_results(label, docs):
    print(f"\n=== {label} (Top 2) ===")
    for d in docs[:2]:
        print("---")
        print(d.page_content[:200].replace('\n',''))

# 질문을 바꾸거나 직접 선정하여 테스트 해보세요
query = "나무-FRP 복합기둥 공법의 장점이 뭐야?"
#query = "간척지에서 쓰는 농업 구조물 기초 기술은?"
#query = "농업기술 잡지에서 소개된 주요 기술 사례는?"

dense_res = dense_retriever.invoke(query)
sparse_res = bm25_retriever.invoke(query)

hybrid_res = hybrid_rrf_retrieve(
    query,
    retrievers=[dense_retriever, bm25_retriever]
)

preview_results("Dense Only", dense_res)
preview_results("Sparse Only", sparse_res)
preview_results("Hybrid (RRF)", hybrid_res)


=== Dense Only (Top 2) ===
---
개선  나무-FRP ‘복합기둥’ 공법②철근배근 ③ 독립기초, 연결부 타설동시진행기존  ‘나무 기둥’ 공법①연결부시공 ②철근배근 ③독립기초타설 ④기둥연결부타설 ①연결부시공   Filament windingGuide railCarriageMandrelFibre spoolResinimpregnation bath*내식성: 부식에 견디는 힘 (
---
글 이학성 농업연구사    농촌진흥청 국립식량과학원 간척지농업연구센터 063-238-548503RDA 포커스간척지	구조물	안전	시공	위한	고성능	복합기둥	공법	개발간척지는 무르고 약한 지반 특성상 구조물을 설치할 때 침하와 시공 난이도 등 다양한 문제가 발생한다. 기존의 ‘콘크리트(PHC) 기둥’ 공법은 침하 안정성은 확보할 수 있으나, 온실 1

=== Sparse Only (Top 2) ===
---
도 폭넓게 적용할 수 있다. 향후 지능형 농장인 스마트팜, 축사, 농촌 주택 등 경량 구조물의 기초기술로도 확장하여 적용할 수 있다. 이 기술은 2025년 5월 특허등록하고, 국제 학술지 카본(Carbon, IF 10.5) 등에 총 4편의 관련 SCI(E) 논문을 게재했다.  복합기둥 공법의 온실시공비구분 나무 기둥 콘크리트 기둥 나무-FRP 
---
글 이학성 농업연구사    농촌진흥청 국립식량과학원 간척지농업연구센터 063-238-548503RDA 포커스간척지	구조물	안전	시공	위한	고성능	복합기둥	공법	개발간척지는 무르고 약한 지반 특성상 구조물을 설치할 때 침하와 시공 난이도 등 다양한 문제가 발생한다. 기존의 ‘콘크리트(PHC) 기둥’ 공법은 침하 안정성은 확보할 수 있으나, 온실 1

=== Hybrid (RRF) (Top 2) ===
---
개선  나무-FRP ‘복합기둥’ 공법②철근배근 ③ 독립기초, 연결부 타설동시진행기존  ‘나무 기둥’ 공법①연결부시공 ②철근배근 ③독립기초타설 ④기둥연결부타설 ①연결부시공   Filament windingGuide r

**관찰 포인트**

- 어떤 방식이 가장 관련성 높은 청크를 포함하는가?  
- Dense-only에서는 누락되지만 Sparse에서는 잡히는 내용이 있는가?  
- Hybrid가 두 방식의 장점을 잘 결합했는가?

#### 정리 

1. **Dense 검색과 Sparse 검색은 서로 보완적인 검색 전략**
2. Hybrid Retrieval(검색 단계 Fusion)로  
   - Recall↑ (빠트리는 문서 줄어듦)  
   - Precision↑ (더 관련성 있는 문서 상위 배치)
3. RRF(검색 단계 Fusion)가 가장 간단하고 안정적인 결합 방식
4. 실무 RAG에서는 대부분 Hybrid Retrieval을 기본값으로 사용

다음 절 **5.3 Reranking & Fusion (Post-Retrieval)** 에서는 이미 검색된 후보를  
Cross-Encoder/LLM/RRF로 **다시 정렬**하여 가장 관련성 높은 문서만 상위에 올리는 방법을 다룹니다.

### 5.3 Reranking & Fusion (Post-Retrieval)

Hybrid Retrieval이 **여러 검색 방식(Dense + Sparse)** 을 결합해 관련 문서를 더 잘 찾도록 도와주는 단계라면,

**Reranking은 검색된 후보들을 다시 평가해 진짜 관련성 높은 문서만 상위에 올리는 단계**입니다.

즉,
- Retrieval = “후보를 찾는 과정”
- Reranking = “후보 중 무엇이 진짜 중요한지 순위를 다시 매기는 과정”

실무에서 Reranking은 **검색 품질 향상에 가장 즉각적인 효과**를 주는 강력한 기법입니다.

#### 5.3.1 왜 Reranking이 중요한가?

Dense 검색 + Sparse 검색을 결합해도 다음 문제는 여전히 남아 있습니다:

1. 의미적으로 비슷한 문서가 여러 개 반환됨 (중복)
2. 비슷하지만 핵심 문서가 아닌 것이 상위에 오기도 함
3. LLM이 요구하는 “정답에 가까운 문서”가 실제 검색 순위 1~2위가 아닐 수 있음

이때 **Reranker**는 검색된 문서와 쿼리를 **다시 비교하여** 
**가장 관련성이 높은 문서만 위로 올려주는 역할**을 합니다.

대표적인 Reranking 방식은 다음과 같습니다.

- **Cross-Encoder Reranking**
- **LLM-grounded Reranking**
- **RRF(Reciprocal Rank Fusion, Fusion 방식)**

이 절에서는 특히 효과가 뛰어난 **Cross-Encoder & RRF** 중심으로 다룹니다

#### 5.3.2 Cross-Encoder 기반 Reranking

Dense/BM25/Hybrid Retrieval은 모두 임베딩을 독립적으로 생성해 벡터 유사도를 비교하는 **bi-encoder 방식**입니다:

- 문서 → 벡터  
- 쿼리 → 벡터  
- 유사도 점수 계산

하지만 bi-encoder는 문서와 질의를 독립적으로 인코딩하기 때문에 
**정교한 관계**까지 잘 반영하지 못하는 경우가 있습니다.

반면 **Cross-Encoder Reranker**는 이렇게 동작합니다:

```
(query, document) → 함께 입력 → 하나의 점수 출력
```

즉, **쿼리와 문서를 동시에 보고 판단**하기 때문에  
관련성 판단이 훨씬 더 정교하고 정확합니다.

대표 모델:
- Cohere Rerank  
- bge-reranker  
- cross-encoder/ms-marco 모델군

In [21]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 1) Reranker 모델 선택

#model_name = "BAAI/bge-reranker-large" # 고성능
model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2" # 경량 버전 (빠름)

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name)
reranker = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
reranker.eval()

# 2) Reranking 함수
def cross_encoder_rerank(query, docs, top_k=None):
    """
    query: 문자열
    docs: Document 객체 리스트 (LangChain)
    top_k: rerank 후 가져올 문서 수
    """

    text_pairs = [(query, d.page_content) for d in docs]

    # Tokenize
    encoded = tokenizer(
        text_pairs,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        scores = reranker(**encoded).logits.squeeze(-1)

    scores = scores.cpu().numpy().tolist()

    # (문서, 점수) 묶어서 정렬
    ranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)

    if top_k:
        ranked = ranked[:top_k]

    return ranked

# 3) 예시 실행
query = "자연어 처리를 배우는 이유는 무엇인가?"

# dense_retriever or BM25 결과 10개를 가져왔다고 가정
candidate_docs = dense_retriever.invoke(query)[:10]

reranked = cross_encoder_rerank(query, candidate_docs, top_k=5)

print("=== Cross-Encoder Reranking 결과 ===")
for doc, score in reranked:
    print(f"[Score: {score:.4f}] {doc.page_content[:200]}")
    print("---")

=== Cross-Encoder Reranking 결과 ===
[Score: 9.0048] 탕으로 열매 모양과 과일 껍질의 털 유무를 판
별할 수 있는 분자 표지를 개발했다. 이외에도 
육종 목표에 부합하는 산함량, 꽃가루 유무 등 
다양한 특성에 관련된 분자 표지를 개발하는 
것이 향후 목표이며, 이를 위해 복숭아 자원의 
다양성을 꾸준히 넓혀갈 것이다. 
열매 모양(원형)
열매 모양(반도형)
과일 껍질 털 있음
과일 껍질 털 없음
복숭아 열매
---
[Score: 8.8307] 근법으로 전환되는 중요한 변화를 의미한다.
  글 이소진 농업연구사    농촌진흥청 국립원예특작과학원 과수기초기반과 063-238-673606
RDA 
포커스
복숭아 유전자원 및 핵심집단
효율성,	다양성	‘꽉’	
복숭아	디지털	육종	기술	개발
14 15
---
[Score: 8.8024] 글 이지혜 농업연구사    농촌진흥청 국립농업과학원 산업곤충과 063-238-294401
RDA 
포커스
누에를 키우는 농업인 양잠(養蠶)은 산업과 함께 변화했다. 과거 우리나라의 양잠은 섬유산업으로서 전성기를 누렸
다. 실크를 만들기 위한 누에고치는 하얗고, 흠이 적으며, 두께가 균일하고, 질긴 특성을 가지는 것이 상품(上品)이었
다. 시대가 흘러 양잠은
---
[Score: 8.7459] 에어로겔은 1~50㎚ 크기의 다공성 입자로 이루어져 있으며, 가볍고 단열성이 매우 뛰어난 물질이다. 최근에는 군용, 
  글 김형권 농업연구관    농촌진흥청 국립원예특작과학원 시설원예연구소 055-580-554005
RDA 
포커스
에어로겔 다겹보온커튼(연동)
에어로겔	다겹보온커튼으로,		
한파	걱정	끝!
에어로겔 다겹보온커튼(단
동
)
기존 다겹보온커튼
---


#### 5.3.3 RRF (Reciprocal Rank Fusion) – Fusion 기반 Reranking

5.2.6.2의 검색 단계 RRF와 달리, 여기서는 검색된 후보들을 다시 정렬하는 Post-Retrieval RRF입니다.  
RRF는 여러 검색기(dense, sparse, multi-query 등)의 결과를  
순위(rank) 기반 점수로 합쳐주는 가장 단순하면서도 매우 강력한 Fusion 기법입니다.

RRF 공식:
```
score = 1 / (k + rank)
```
- rank: 검색된 문서의 순위(0-based 또는 1-based)
- k: 일반적으로 60 정도로 고정  
→ 높은 순위일수록 점수가 크고, 여러 검색기에서 고순위를 받으면 점수가 합쳐지며 최종적으로 상위에 노출됨.

#### 장점
- 구현이 매우 간단함  
- Dense / BM25 / Multi-Query 등 어떤 조합에도 잘 작동  
- Cross-Encoder가 없어도 Recall이 크게 개선됨  
- 대규모 실무 환경에서 **표준 Fusion 기법**으로 널리 사용됨  

LangChain 최신 버전에서는 기존의 `EnsembleRetriever`가 삭제되었기 때문에,  
직접 RRF 스코어를 계산하는 방식으로 구현한다.

In [22]:
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document

# 1) Sparse Retriever
sparse_retriever = bm25_retriever

# 2) Dense Retriever (예: FAISS retriever)
dense_retriever  # 위에서 이미 생성

# 3) RRF 함수 구현
def rrf_fusion(retrievers, query, k=60):
    scores = {}
    
    for retriever in retrievers:
        results = retriever.invoke(query)
        
        for rank, doc in enumerate(results):
            doc_id = doc.metadata.get("id", id(doc))
            # RRF 점수 누적
            scores.setdefault(doc_id, {"doc": doc, "score": 0})
            scores[doc_id]["score"] += 1 / (k + rank + 1)

    # 점수 기준 정렬 후 문서만 반환
    fused = sorted(scores.values(), key=lambda x: x["score"], reverse=True)
    return [item["doc"] for item in fused]

class HybridRRF:
    """간단한 RRF 기반 하이브리드 retriever"""
    def __init__(self, retrievers, k=60):
        self.retrievers = retrievers
        self.k = k

    def _retrieve(self, query):
        return rrf_fusion(self.retrievers, query, k=self.k)

    def invoke(self, query):
        return self._retrieve(query)

    def get_relevant_documents(self, query):
        return self._retrieve(query)

# 4) 인스턴스 생성 및 실행
hybrid_rrf = HybridRRF([dense_retriever, sparse_retriever], k=60)
hybrid_results = hybrid_rrf.invoke(query)

print("RRF 결과 Top 3:")
for r in hybrid_results[:3]:
    print(r.page_content[:200])
    print("---")


RRF 결과 Top 3:
탕으로 열매 모양과 과일 껍질의 털 유무를 판
별할 수 있는 분자 표지를 개발했다. 이외에도 
육종 목표에 부합하는 산함량, 꽃가루 유무 등 
다양한 특성에 관련된 분자 표지를 개발하는 
것이 향후 목표이며, 이를 위해 복숭아 자원의 
다양성을 꾸준히 넓혀갈 것이다. 
열매 모양(원형)
열매 모양(반도형)
과일 껍질 털 있음
과일 껍질 털 없음
복숭아 열매
---
에어로겔은 1~50㎚ 크기의 다공성 입자로 이루어져 있으며, 가볍고 단열성이 매우 뛰어난 물질이다. 최근에는 군용, 
  글 김형권 농업연구관    농촌진흥청 국립원예특작과학원 시설원예연구소 055-580-554005
RDA 
포커스
에어로겔 다겹보온커튼(연동)
에어로겔	다겹보온커튼으로,		
한파	걱정	끝!
에어로겔 다겹보온커튼(단
동
)
기존 다겹보온커튼
---
글 이학성 농업연구사    농촌진흥청 국립식량과학원 간척지농업연구센터 063-238-548503
RDA 
포커스
간척지	구조물	안전	시공	위한	
고성능	복합기둥	공법	개발
간척지는 무르고 약한 지반 특성상 구조물을 설치할 때 침하와 시공 난이도 등 다양한 문제가 발생한다. 기존의 ‘콘
크리트(PHC) 기둥’ 공법은 침하 안정성은 확보할 수 있으나, 온실 1
---


#### 5.3.4 LLM-grounded Reranking

최근 트렌드는 **LLM이 검색된 문서들을 직접 판단하도록 하는 방식**입니다.

예:
```
“다음 문서 중 이 질문에 가장 relevant한 순서로 정렬해줘”
```

장점:
- LLM reasoning을 활용해 관련성 판단이 더 정교함  
단점:
- 비용 증가  
- 속도 느림

#### Reranking 실습: 전체 흐름 비교

In [23]:
query = "농업 기술 지원 기준 알려줘"

dense_res = dense_retriever.invoke(query)
sparse_res = sparse_retriever.invoke(query)
rrf_res = rrf_fusion([dense_retriever, sparse_retriever], query)
rerank_res = cross_encoder_rerank(query, dense_res, top_k=5)
rerank_res = list(map(lambda x : x[0], rerank_res)) # 점수를 제거하고 문서만 남김

def show(label, docs, n=3):
    print(f"\n=== {label} ===")
    for d in docs[:n]:
        print(d.page_content[:200])
        print("---")

show("Dense Only", dense_res)
show("Sparse Only", sparse_res)
show("Hybrid RRF", rrf_res)
show("Cross-Encoder Rerank", rerank_res)


=== Dense Only ===
도 폭넓게 적용할 수 있다. 향후 지능형 농장
인 스마트팜, 축사, 농촌 주택 등 경량 구조물
의 기초기술로도 확장하여 적용할 수 있다. 이 
기술은 2025년 5월 특허등록하고, 국제 학술
지 카본(Carbon, IF 10.5) 등에 총 4편의 관련 
SCI(E) 논문을 게재했다. 
 복합기둥 공법의 온실시공비
구분 나무 기둥 콘크리트 기둥 나무-FRP 
---
2025. 11+12월호 
vol.695
농업기술
AGRICULTURAL TECHNOLOGY
간척지 구조물 안전시공
고성능 복합기둥 공법개발로 
다양한 지형에 폭넓게 적용
연두색 누에고치 품종
‘연녹잠’의 기능성과 가능성
혈당과 장건강까지 챙기는
‘도담쌀’
재배 안정성이 강화된
장원형 쌀 ‘케이롱’
축산 분야 
한기웅 명인
RDA 포커스
알쓸신농
추천! K
---
글 이학성 농업연구사    농촌진흥청 국립식량과학원 간척지농업연구센터 063-238-548503
RDA 
포커스
간척지	구조물	안전	시공	위한	
고성능	복합기둥	공법	개발
간척지는 무르고 약한 지반 특성상 구조물을 설치할 때 침하와 시공 난이도 등 다양한 문제가 발생한다. 기존의 ‘콘
크리트(PHC) 기둥’ 공법은 침하 안정성은 확보할 수 있으나, 온실 1
---

=== Sparse Only ===
스마트폰으로 QR코드를 찍으면 ‘농업과학도서관’으로 연결되어 ‘농업기술’과 ‘농업기술길잡이’ 등
농촌진흥청에서 발행하는 다양한 책자를 만날 수 있습니다. 포털사이트에 ‘농업과학도서관’을 검색해보세요.
[54875] 전북특별자치도 전주시 덕진구 농생명로 300(중동)   T.1544-8572   F.063-238-1766   www.rda.go.kr
2026
---
근법으로 전환되는 중요한 변화를 의미한다.
  글 이소진 농업연구사    농촌진흥청 국립원예특작과학원 과수기초기반과 063-238-673606
RDA 
포커스
복숭아 유전자원 및 핵심집단
효율성,	다양성	‘꽉’	
복숭아	디지털	육종	기술	개발
14 

#### 정리

1. Retrieval은 “후보를 찾는 과정”이고 Reranking은 “그 후보들의 진짜 중요도를 다시 매기는 과정”
2. Cross-Encoder 기반 Reranking은 가장 정교하고 정확함
3. RRF Fusion은 간단하면서도 매우 강력한 정렬 전략
4. Hybrid + Rerank 조합은 엔터프라이즈 RAG에서 사실상 표준

### 5.4 Context Filtering & Compression (Post-Retrieval)

**Context Filtering & Compression은 검색된 문서 중 실제로 필요한 부분만 선별하고, LLM이 처리하기 좋은 형태로 재구성하는 단계입니다.**  
Retrieval과 Reranking을 거친 뒤에도, 검색된 문서에는 다음과 같은 문제가 남아 있을 수 있습니다:

- 관련성이 낮은 청크가 일부 포함됨  
- 비슷한 내용이 중복되어 맥락이 불필요하게 길어짐  
- 중요한 정보가 긴 문서 내부에 묻혀 있음  
- 전체 길이가 LLM의 컨텍스트 창을 초과하거나, 모델의 주의(attention)가 분산됨  

이러한 이유로 Advanced RAG에서는 **검색된 문서를 더 정제하여 LLM이 활용하기 좋은 형태로 만드는 과정**이 필요하며,  
이를 **Context Filtering & Compression**이라고 부릅니다.

#### 5.4.1 왜 Filtering & Compression이 중요한가?

LLM이 검색된 문서를 그대로 처리하기 어렵다는 점에서 출발합니다.

##### ① 컨텍스트 창 제한  
대형 모델은 컨텍스트 창이 넓어진 반면, 일반적으로는 제한이 존재하며 비용도 증가합니다.  
검색 결과를 모두 넣으면 오히려 정보 밀도가 낮아지고 답변 품질이 떨어질 수 있습니다.

##### ② 불필요한 정보는 모델의 판단을 흐릴 수 있음  
중복 청크, 소제목·부록, 잡음(noisy text) 등이 많을수록  
모델의 attention이 분산되며 핵심 내용을 찾아내기 어려워집니다.

##### ③ 중요한 정보가 길이에 묻혀 reasoning이 실패할 수 있음  
검색된 문서가 항상 원하는 답을 직접 제시하는 것은 아니며, LLM은 필요한 부분을 추론해야 합니다.  
이때 문맥이 과도하게 길면 핵심 근거를 인식하지 못해 추론이 실패할 수 있습니다.

Filtering & Compression은 이러한 문제를 줄이고  
**LLM이 실제 답변에 필요한 정보만 명확하게 바라볼 수 있도록 돕는 과정**입니다.

#### 5.4.2 Context Filtering 기법

##### ✔ (1) Relevance Filtering  
검색된 문서들 중에서 “진짜 질문과 관련된 문서만 남기기”

방법:
- 단순 BM25 매칭 기반 필터  
- 임베딩 유사도 기반 cutoff  
- LLM에게 직접:  
  ```
  쿼리와 관련 없는 문서를 골라 제거하라
  ```

##### ✔ (2) Deduplication (중복 제거)  

같은 내용을 가진 문서가 여러 개 있을 경우 제거.

- 텍스트 유사도 기반  
- 임베딩 유사도 기반  
- Cosine similarity threshold 사용

##### ✔ (3) Segment-level Filtering  
문서 전체가 아니라 문서의 일부만 필요할 경우:

- 슬라이딩 윈도우 기반 필터  
- 문장 단위 임베딩 후 중요 문장만 선택  
- 문서 내부에서 “답변에 중요한 부분만 추출”

#### 5.4.3 Context Compression 기법

##### ✔ (1) Extractive Compression  
문서 중 **핵심 문장만 추출**  
- 문장 임베딩 기반 scoring  
- 범주형 분류 기반 extraction  
- LLM 기반 extractive summarization

##### ✔ (2) Abstractive Compression  
LLM이 문서를 이해한 뒤 **요약된 새로운 문장을 생성**

- "문서 내용을 절반 이하 길이로 요약하라"  
- “이 질문에 필요한 핵심 정보만 남겨라”  

##### ✔ (3) LLM-grounded Compression  
LLM이 문서를 읽고 직접 relevance scoring 을 수행:

질문 + 문서 →  
**이 문서가 필요한지 판단 → 필요하다면 중요한 부분만 압축**

→ 최근 가장 강력한 방식

#### 실습: Contextual Compression Retriever

기본 Retriever는 단순히 유사도가 높은 청크를 여러 개 가져오기만 합니다.  
하지만 이 결과에는 중복된 내용, 불필요한 배경 설명, 질문과 상관없는 정보가 섞여 있을 때가 많습니다.   

이를 해결하기 위해 RAG에서는 검색 후(Post-retrieval) 단계에서 문서를 정제합니다.  
그 대표적인 방식이 Contextual Compression Retriever 입니다.  

이를 활용하여 간단한 Context Filtering & Compression 실습을 진행 하겠습니다.  

#### 실습: Post-Retrieval 문맥 압축 적용하기

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document

# LLM 설정: 문서를 "압축 요약"하는 역할만 담당
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 문서 압축용 프롬프트
# - 질문과 관련된 정보만 남기고 불필요한 서술은 제거하도록 안내
compress_prompt = ChatPromptTemplate.from_template("""
다음 문서 내용을 보고, 질문에 답하는 데 필요한 핵심 정보만 간단히 요약해 주세요.

질문: {query}

문서:
{content}

출력 규칙:
- 질문과 직접 관련된 내용만 3~5줄로 요약
- 배경 설명, 수식어, 문서 전체 요약은 제외
""")

compress_chain = compress_prompt | llm


def compressed_retrieve(dense_retriever, query: str, k: int = 4):
    """
    ContextualCompressionRetriever를 직접 구현한 간단 버전.
    1) Retriever가 가져온 원문을
    2) LLM을 이용해 '질문에 필요한 부분만' 압축 요약하여 반환한다.

    왜 필요한가?
    - 검색된 문서는 길고 불필요한 정보가 많을 수 있다.
    - LLM은 긴 문맥에서 중요한 정보를 찾지 못해 오류가 발생하기 쉽다.
    → 따라서 '검색 후 압축' 과정이 성능 향상에 중요하다.
    """
    
    docs = dense_retriever.invoke(query)

    compressed_docs = []

    for d in docs:
        # LLM이 문서에서 "질문과 관련 있는 핵심 부분만" 추출하도록 요청
        response = compress_chain.invoke({
            "query": query,
            "content": d.page_content
        })
        summary = response.content

        # 기존 metadata는 유지한 채, 압축된 내용만 교체
        compressed_docs.append(
            Document(page_content=summary, metadata=d.metadata)
        )

    return compressed_docs[:k]


# 실행 예시
query = "농업 기술 보급 기준은 무엇인가?"
compressed_docs = compressed_retrieve(dense_retriever, query, k=4)

print("압축된 문서 개수:", len(compressed_docs))
for d in compressed_docs:
    print("---")
    print(d.page_content[:300])


압축된 문서 개수: 4
---
농업 기술 보급 기준은 초기 비용이 적고, 내식성과 강도가 우수한 소재를 활용하여 안정적인 시공이 가능한 기술을 개발하는 것이다. 농촌진흥청은 나무 기둥에 FRP(섬유강화플라스틱)를 결합한 '복합기둥' 공법을 통해 간척지에서의 구조물 설치 시 발생하는 문제를 해결하고자 한다.
---
농업 기술 보급 기준은 복합기둥 공법을 통해 다양한 지형에 폭넓게 적용할 수 있는 기술로, 시공비를 75% 절감하고 안정성을 확보하는 것을 포함한다. 이 기술은 2025년 5월 특허 등록 예정이며, 20년 이상 유지 가능한 내부식성을 갖추고 있다.
---
농업 기술 보급 기준은 식물특허 보호에 따라 원료 사용 시 한국농업기술진흥원을 통한 기술이전 절차를 반드시 거쳐야 하며, 정식 권리 이전이 이루어져야 종자를 공급받아 사용할 수 있다.
---
농업 기술 보급 기준은 다양한 지형에 적용 가능한 고성능 복합기둥 공법 개발, 기능성과 가능성을 갖춘 품종의 재배 안정성 강화, 그리고 혈당과 장건강을 고려한 쌀 품종의 개발 등을 포함한다.


**관찰 포인트**
- 검색된 문서보다 압축된 문서의 길이가 크게 줄어 있는가?  
- 질문과 직접 관련 없는 문서가 제거되었는가?  
- 불필요한 서론/부록/표 등이 사라졌는가?

#### 실습: 청크 중복 제거(Deduplication)로 문서 품질 개선

문서를 청크 단위로 분할하면, 간혹 다음과 같은 문제가 발생합니다:

- 동일한 문단이 여러 번 반복되어 있음  
- 표현만 조금 다를 뿐, 사실상 같은 내용을 담은 청크가 여러 개 생성됨  

이러한 중복 청크는 **Retriever의 판단을 흐리고, LLM의 컨텍스트 창을 불필요하게 소모**합니다.  
따라서 Post-Retrieval 단계에서 중복을 제거하면 **검색 품질과 모델 효율이 모두 개선**됩니다.

In [25]:
import numpy as np

def deduplicate_chunks(chunks, threshold=0.92):
    """
    의미가 거의 같은 청크(텍스트 조각)를 제거해서,
    비슷한 내용이 여러 번 들어가는 것을 막는 함수입니다.

    chunks: List[str]
        임베딩을 만들고 싶은 텍스트 청크 리스트
    threshold: float
        코사인 유사도 임계값 (0~1).
        두 청크의 유사도가 threshold보다 크면 "거의 같은 내용"으로 보고 중복으로 제거합니다.
    """

    if not chunks:
        return []

    # 1) 문서 임베딩 계산
    #    - embeddings.embed_documents(chunks)는 각 청크를 벡터로 바꿔줌
    vectors = np.array(embeddings.embed_documents(chunks))

    # 2) 코사인 유사도 계산을 빠르게 하기 위해 벡터를 정규화(L2 norm = 1)해 둡니다.
    #    - 코사인 유사도 = (정규화된 벡터끼리의 내적)
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    vectors_norm = vectors / (norms + 1e-12)  # 0으로 나누는 오류 방지용 작은 값 추가

    keep_indices = []      # 최종적으로 유지할 청크의 인덱스
    removed = set()        # 이미 "중복"으로 판정되어 제거된 인덱스

    n = len(chunks)

    for i in range(n):
        # 이미 다른 청크와 너무 비슷해서 제거된 경우 건너뜀
        if i in removed:
            continue

        keep_indices.append(i)

        # i보다 뒤에 있는 청크들과의 유사도만 비교 (i 이전은 이미 비교 완료)
        # 코사인 유사도 = 내적(dot product) (벡터를 정규화해두었기 때문에 가능)
        sims = vectors_norm[i] @ vectors_norm[i+1:].T  # shape: (n-i-1,)

        for offset, sim in enumerate(sims, start=1):
            j = i + offset
            if sim > threshold:
                removed.add(j)

    # 인덱스를 기반으로 실제 텍스트 청크 리스트를 다시 만듦
    return [chunks[i] for i in keep_indices]


# 사용 예시
deduped_chunks = deduplicate_chunks(structured_chunks, threshold=0.92)

print(f"중복 제거 전 청크 개수: {len(structured_chunks)}")
print(f"중복 제거 후 청크 개수: {len(deduped_chunks)}")


중복 제거 전 청크 개수: 26
중복 제거 후 청크 개수: 26


#### 실습 : 검색 결과 중복 제거 (Post-Retrieval Deduplication)

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

def dedupe_after_retrieval(docs, threshold=0.92):
    """
    검색된 문서들(docs) 중에서 서로 매우 유사한 청크를 제거합니다.
    - threshold 이상으로 유사하면 중복으로 간주
    - LLM에 전달되는 중복/노이즈를 줄여 reasoning 실패를 예방
    """

    # 청크 텍스트를 벡터로 변환
    texts = [d.page_content for d in docs]
    vectors = embeddings.embed_documents(texts)

    keep = []
    removed = set()

    for i in range(len(docs)):
        if i in removed:
            continue

        keep.append(docs[i])

        for j in range(i + 1, len(docs)):
            sim = cosine_similarity([vectors[i]], [vectors[j]])[0][0]

            # 유사도가 threshold 이상이면 중복으로 판단
            if sim > threshold:
                removed.add(j)

    return keep


# 테스트: retriever로 문서 검색 → 중복 제거
query = "연녹잠 누에고치 기능성 요약해줘"

retrieved_docs = dense_retriever.invoke(query)
print("검색된 문서 수:", len(retrieved_docs))

deduped_docs = dedupe_after_retrieval(retrieved_docs, threshold=0.92)
print("중복 제거 후 문서 수:", len(deduped_docs))

for d in deduped_docs:
    print("-" * 40)
    print(d.metadata.get("id"), d.page_content[:250])


검색된 문서 수: 4
중복 제거 후 문서 수: 4
----------------------------------------
6 하여 연녹잠 고치로 스마트섬유를 만들어 볼 
수 있을 것이다. 피부미용에도 효과가 좋았다. 
연녹잠 홍잠(고치 짓기 전 성숙한 누에를 찐 
것)을 섭취한 실험쥐에서 멜라닌 생성이 억제
되고, 피부밝기가 41% 개선되었다. 이와 같
이 연녹잠은 기능성 누에 품종으로 식의약 소
재 외에도 화장품, 바이오소재 등 다양한 분야
에 활용될 수 있을 것으로 보인다.
‘연녹잠’, 우수 누에 품종의 보급
다양한 잠재력을 가진 연녹잠은 우수 누에 품
종으로 지
----------------------------------------
5 연녹잠 추출물연녹잠 추출물
콜레스테롤 콜레스테롤 
효과적으로 감소효과적으로 감소
‘연녹잠’누에고치 추출물의 
콜레스테롤 조절 효과
연녹잠 누에고치 추출물은 콜레스테롤 대사 
개선에 효과를 보였다. 고지방식이로 비만을 
유도한 마우스 모델에 투여하였을 때, 나쁜 콜
레스테롤로 알려진 저밀도 지단백(LDL) 콜레
스테롤의 혈중농도가 26% 감소했다. 이는 콜
레스테롤 치료제로 알려진 스타틴(Statin)과 
유사한 수준이다. 간 조직의 콜레스테롤 
----------------------------------------
4 글 이지혜 농업연구사    농촌진흥청 국립농업과학원 산업곤충과 063-238-294401
RDA 
포커스
누에를 키우는 농업인 양잠(養蠶)은 산업과 함께 변화했다. 과거 우리나라의 양잠은 섬유산업으로서 전성기를 누렸
다. 실크를 만들기 위한 누에고치는 하얗고, 흠이 적으며, 두께가 균일하고, 질긴 특성을 가지는 것이 상품(上品)이었
다. 시대가 흘러 양잠은 기능성 산업으로 전환기를 맞는다. 누에 분말은 혈당 강하 소재로, 누에고치 추출물은 기억
----------------------------------------
23 난방비 절감 및난방비 절감 및
재배환경 개선,재배환경 개선,
탄소중립까지 가능해탄소

#### 정리

1. Retrieval 결과를 그대로 LLM에 전달하는 것이 항상 최적은 아님.  
2. **Filtering**: 관련 없는 문서를 제거하거나, 사용 목적에 맞지 않는 청크를 제외하는 단계  
3. **Compression**: 필요한 문서를 유지하되, LLM이 활용하기 좋은 형태로 **정보를 압축(요약, 핵심 추출, 재구성)** 하는 단계  
4. LangChain의 Contextual Compression Retriever는  
   Filtering + Compression을 자동으로 수행  
5. Filtering & Compression은 long-context 문제를 해결하는 핵심 전략  

다음 절 **5.4 Generator 단계 최적화**에서는, 검색된 문맥을 기반으로 LLM이 더 정확하게 답변하도록  
**프롬프트 구조, citation 방식, output constraints** 등을 다룹니다.


### 5.5 Generation 단계 최적화

Retrieval / Reranking / Filtering / Compression을 거친 후  
마지막 단계는 **LLM이 실제로 답변을 생성하는 Generation 단계**입니다.

많은 개발자가 “검색 성능이 좋아지면 답변도 좋아진다”고 생각하지만,  
**Generation 단계 자체가 최적화되지 않으면 여전히 다음 문제가 발생할 수 있습니다.**

- 검색된 문서를 제대로 활용하지 못함  
- 답변이 검색 문맥과 동떨어짐 (hallucination)  
- 불필요하게 긴 답변  
- citation(출처 근거) 없음  
- JSON 등 구조화된 출력 실패  

따라서 Advanced RAG에서는 **검색된 컨텍스트를 LLM이 가장 잘 소비하도록 Prompt 구조를 최적화하는 과정**이 반드시 필요하다.

#### 5.5.1 Generation 단계가 어려운 이유

##### ✔ 1) LLM은 문서를 “그대로 반영”하지 않는다  
 - LLM은 문서의 구조적 의미(조건·예외·표 등)를 정확히 파악하는 데 한계가 있어 내용의 핵심을 놓치기 쉽다.  
 - 특히 문서가 길어질수록 attention이 희석되어 일부 정보만 단편적으로 반영된다.  
 - 이로 인해 실제 문서의 기준이나 요건을 잘못 이해하거나 누락하는 오류가 발생한다.

##### ✔ 2) 컨텍스트와 질문 사이의 연결을 모델이 자동으로 맞추지 못함  
 - LLM은 질문에 맞는 답변 패턴을 생성하려는 성향이 강해 문서 내용과 질문의 alignment가 쉽게 어긋난다.  
 - 문서에 없는 내용을 일반 지식을 기반으로 보완하거나, 질문과 무관한 문서의 부분을 가져오는 “semantic drift”가 발생한다.  
 - 프롬프트로 제약을 명확히 주지 않으면 문서 기반 답변이 아니라 추정 기반 답변을 만들어내기 쉽다.

##### ✔ 3) Hallucination은 Generation 단계에서 주로 발생  
 - 문서가 부족하거나 질문이 모호하면 LLM은 빈 부분을 상식이나 추론으로 채우려 하면서 새로운 내용을 만들어낸다.  
 - 문서 길이가 길거나 모델 규모가 작을수록 문맥 유지가 어려워져 오답 생성 확률이 높아진다.  
 - 이 때문에 검색이 정확해도 생성 단계에서 잘못된 정보로 답변이 왜곡될 수 있다.

##### ✔ 4) 개발자가 원하는 출력 형태(JSON, bullet point 등)를 LLM이 항상 지키기 어렵다  
- LLM은 정해진 포맷을 따르는 것보다 자연스러운 문장을 생성하는 방향으로 확률이 치우쳐 있다.  
- 출력 구조가 복잡해질수록 형식을 깨거나 일부 필드를 누락하는 문제가 발생한다.  
- 이는 RAG에서 API 응답 형태나 strict format이 필요한 경우 큰 장애로 이어질 수 있다.

#### 5.5.2 Generation 최적화의 핵심 전략

Advanced RAG에서 Generator 최적화는 크게 4가지로 나뉜다.

- > ① Prompt 구조 최적화  
- > ② Citation / Attribution 기반 생성  
- > ③ Output Constraint (형식 강제)  
- > ④ Fail-safe 답변 전략 (안전성 향상)

#### 5.5.3 Prompt 구조 최적화

좋은 RAG Prompt의 구조는 아래 형태를 따른다:

```
[역할 정의 Role]
당신은 문서 기반 질문응답 시스템입니다.
문서에 기반하지 않은 내용은 절대 답변하지 마십시오.

[질문 Question]
{query}

[컨텍스트 Context]
{context}

[답변 지침 Instructions]
- 문서에 있는 내용만 답변할 것
- 문서에 없는 정보는 "문서에 정보가 없습니다"라고 말할 것
- 핵심 요약 후 필요한 경우 상세 설명 제공
```

#### 실습: 프롬프트 템플릿 구성

In [27]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = PromptTemplate.from_template("""
당신은 문서 기반 QA 시스템입니다.
아래 문맥(Context)에 기반해서만 답변하세요.
문맥에 없는 내용은 절대 생성하지 마십시오.

[질문]
{query}

[문맥]
{context}

[출력 형식]
- 핵심 결론을 먼저 말한다.
- 문서에 근거한 사실만 말한다.
- 필요한 경우 문서의 일부 문장을 인용한다.

답변:
""")

#### 5.5.4 Citation / Source Attribution

사용자 신뢰도를 높이기 위해  
**문서의 어느 부분에서 답을 가져왔는지** 표시하도록 할 수도 있습니다.

예시:

```
지원 기준은 다음과 같습니다:
- 기술 검토 절차 필수 (출처: 문서 2번 청크)
- 관련 기관 승인 필요 (출처: 문서 4번 청크)
```

#### 실습: Citation 포함 답변 생성

In [28]:
dense_retriever.invoke(query)

[Document(id='91b185cf-fd6b-4656-9f29-bebe9b1c10b6', metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20251120203111', 'source': 'data/농업기술.pdf', 'total_pages': 16, 'page': 5, 'page_label': '6', 'id': 6}, page_content='하여 연녹잠 고치로 스마트섬유를 만들어 볼 \n수 있을 것이다. 피부미용에도 효과가 좋았다. \n연녹잠 홍잠(고치 짓기 전 성숙한 누에를 찐 \n것)을 섭취한 실험쥐에서 멜라닌 생성이 억제\n되고, 피부밝기가 41% 개선되었다. 이와 같\n이 연녹잠은 기능성 누에 품종으로 식의약 소\n재 외에도 화장품, 바이오소재 등 다양한 분야\n에 활용될 수 있을 것으로 보인다.\n‘연녹잠’, 우수 누에 품종의 보급\n다양한 잠재력을 가진 연녹잠은 우수 누에 품\n종으로 지정되어 농촌진흥청에서 매년 누에씨\n를 지자체 유관기관과 협력하여 보급하고 있\n다. 농가수요에 따른 생산계획 수립으로 연녹\n잠과 같은 우수 누에 품종 보급을 확대하고, \n국내 기능성 양잠산업의 기반을 강화하기 위\n한 노력을 지속할 계획이다. \n농촌진흥청 육종 누에품종별 누에고치 색상\n‘연녹잠’ 누에고치 추출물 콜레스테롤 강하효과 동물실험\n대조군      ATV      연녹잠\n(양성 대조군)\n(A) 혈액 총콜레스테롤 (B) 혈액 HDL-콜레스테롤 (C) 혈액 LDL-콜레스테롤 (D) 간 콜레스테롤\n대조군      ATV      연녹잠\n(양성 대조군)\n대조군      ATV      연녹잠\n(양성 대조군)\n대조군      ATV      연녹잠\n(양성 대조군)\n백옥잠 골든실크 주황잠 연녹잠\n2025. 11 + 12  농촌진흥청  농업기술 \n5'),
 Document(id='ef30c779-b059-4992-8c32-49c9aabd12a7', met

In [29]:
def format_context_with_ids(docs):
    combined = ""
    for i, d in enumerate(docs):
        combined += f"[문서{i}]\n{d.page_content}\n\n"
    return combined

query = "연녹잠 누에고치가 콜레스테롤을 낮춘다는 연구 결과 알려줘"

docs = dense_retriever.invoke(query)
formatted_context = format_context_with_ids(docs)

response = llm.invoke(prompt.format(query=query, context=formatted_context))
print(response.content)

연녹잠 누에고치 추출물은 콜레스테롤 대사를 개선하여 혈중 나쁜 콜레스테롤인 저밀도 지단백(LDL) 콜레스테롤 농도를 26% 감소시키는 효과를 보였습니다. 이는 콜레스테롤 치료제로 알려진 스타틴과 유사한 수준입니다. 또한, 연녹잠 고치는 간에서 콜레스테롤을 배출하는 기전을 활성화하여 간 콜레스테롤이 27% 감소하였습니다. 문서에서는 "연녹잠 고치 추출물은 콜레스테롤 대사 개선에 효과를 보였다"라고 언급하고 있습니다.


#### 5.5.5 Output Constraint (JSON 구조 강제)

LLM이 정확한 구조화 출력을 해야 하는 경우:

- API 응답 구조 맞추기
- 시스템 간 연동
- 대시보드용 structured data  

등에서 필수적입니다.

#### 예: JSON으로 답변 강제

In [30]:
json_prompt = PromptTemplate.from_template("""
아래 문맥에 기반하여 질문에 답하십시오.
출력은 반드시 JSON으로만 생성하십시오.
기타 설명 금지.

문맥:
{context}

질문:
{query}

JSON 형식:
{{
    "summary": "...",
    "details": ["...", "..."],
    "confidence": "high | medium | low"
}}
""")

실습:

In [31]:
resp = llm.invoke(json_prompt.format(query=query, context=formatted_context))
print(resp.content)

{
    "summary": "연녹잠 누에고치 추출물이 콜레스테롤 대사를 개선하여 LDL 콜레스테롤의 혈중농도를 26% 감소시켰다는 연구 결과가 있습니다.",
    "details": [
        "연녹잠 추출물을 고지방식이로 비만을 유도한 마우스 모델에 투여한 결과, LDL 콜레스테롤이 26% 감소했습니다.",
        "연녹잠은 스타틴과 유사한 수준의 혈중 콜레스테롤 치료 효과를 보이며, 간 콜레스테롤 함량이 27% 감소했습니다."
    ],
    "confidence": "high"
}


#### 5.5.6 Fail-safe 답변 전략

문서에 정보가 없을 때는 다음을 강제해야 한다:

```
문서에 정보가 없습니다. (hallucination 방지)
```

#### 참고: Fail-safe 포함 프롬프트

In [32]:
safeguard_prompt = PromptTemplate.from_template("""
문서에 근거하지 않은 정보는 만들어내지 마십시오.
문서에 답이 없으면 다음과 같이 말하십시오:
"문서에 해당 정보가 없습니다."

문서:
{context}

질문:
{query}
""")

#### 5.5.7 전체 Generation 흐름 실습

In [33]:
query = "농업 기술 보급 기준이 뭐야?"

docs = dense_retriever.invoke(query)
context = format_context_with_ids(docs)

response = llm.invoke(
    prompt.format(query=query, context=context)
)

print("=== 최종 답변 ===")
print(response.content)

=== 최종 답변 ===
농업 기술 보급 기준에 대한 구체적인 내용은 문맥에 포함되어 있지 않습니다. 따라서 별도의 답변을 제공할 수 없습니다.


관찰 포인트:

- 문서에 기반한 답변인가?  
- 문서에 없는 내용을 추가로 말하지 않는가?  
- 핵심 내용을 먼저 말하는가?  

#### 5.5.8 정리

1. 검색 품질이 좋아도 Generation 단계가 최적화되지 않으면 RAG 전체 품질이 떨어짐  
2. Prompt 구조 최적화는 hallucination 방지의 핵심  
3. Citation/Output constraints는 실무에서 필수  
4. Fail-safe 전략으로 안전성 강화  
5. Retrieval 품질 + Generation 최적화가 함께 적용될 때 RAG 성능이 극적으로 향상  

다음 절 에서는 실무에서 매우 중요한 **Retrieval 품질 평가**, **Generation 품질 평가**,  
그리고 Recall@k, nDCG 등 검증 지표와 평가 실습을 다룹니다.

## 6. RAG 성능 평가 (Evaluation)  

**평가(Evaluation)는 RAG에서 가장 중요하면서도, 실제 구현 과정에서는 종종 간과되는 핵심 단계입니다.**  

검색 품질이 충분한지, 생성이 문서 기반으로 이루어지는지, 그리고 전체 파이프라인이 의도한 대로 작동하는지를  

**정량적·정성적 기준으로 체계적으로 검증하는 과정이 반드시 필요합니다.**

<img src="image/eval.png" width="380">

RAG 평가는 크게 두 영역으로 나눌 수 있습니다.

1. **Retrieval Evaluation (검색 품질 평가)**  
2. **Generation Evaluation (생성 품질 평가)**  

이 절에서는 두 평가 방법의 이론적 배경과 실제 적용 절차를 함께 다룹니다.

### 6.1 왜 RAG 평가가 중요한가?

다음과 같은 상황은 모두 **평가가 부실할 때** 흔히 발생하는 문제들입니다.

- 벡터 유사도 기반 검색만 사용하다 보니, 실제로 중요한 문서가 반복적으로 검색되지 않음  
- 검색은 잘 되었는데 LLM이 문서를 근거로 답하지 않고 자체 추론으로 대답함  
- Chunk 크기나 슬라이딩 윈도우 전략을 바꿔도 어떤 설정이 더 효과적인지 판단할 근거가 없음  
- Query Rewriting/Expansion을 적용했지만, 오히려 Recall이 떨어져 성능이 악화됨  
- BM25·Vector·Hybrid Retrieval 실험을 해도 정량적 지표 없이 “감”으로 방식이 선택됨  

따라서 **RAG 파이프라인을 안정적으로 개선하기 위해서는 체계적 평가가 필수적입니다.**

### 6.2 Retrieval Evaluation (검색 품질 평가)

#### 핵심 지표

| 지표 | 설명 |
|------|------|
| **Recall@K** | 정답 문서가 상위 K개 검색 결과 안에 포함되는 비율 |
| **Precision@K** | 상위 K개 검색 결과 중 실제로 정답에 해당되는 문서 비율 |
| **MRR (Mean Reciprocal Rank)** | 정답 문서가 몇 번째에 등장했는지의 역수 평균 |
| **nDCG (Normalized Discounted Cumulative Gain)** | 순위의 품질을 점수 기반으로 평가하는 지표 |

RAG에서 가장 중요한 지표는 **Recall@K** 입니다.  
정답 문서가 검색 단계에서 확보되지 않으면, LLM이 아무리 뛰어나더라도 올바른 답변을 생성하기 어렵습니다.

#### 6.2.1 Retrieval 평가를 위한 데이터 구조

Retrieval 평가는 다음처럼 구성된 데이터셋을 필요로 한다.

```python
test_cases = [
    {
        "query": "농업 기술 보급 기준은?",
        "answer_doc_ids": [3, 7]   # 정답이 들어있는 문서(청크)의 ID
    },
    ...
]
```

각 질문마다 정답이 포함된 청크 ID를 지정해 주면 됩니다.

#### 실습: Recall@K 계산하기

##### ① 검색 함수 준비

In [34]:
def retrieve_ids(retriever, query, k=5):
    docs = retriever.invoke(query)
    return [d.metadata.get("id") for d in docs[:k]]

※ 전제: 청크 생성 과정에서 metadata에 "id": index 형태로 저장되어 있다고 가정합니다.

In [35]:
test_cases = [
    {
        "query": "연녹잠 누에고치의 기능성과 활용 가능성을 알려줘",
        "answer_doc_ids": [4, 5, 6]
    },
    {
        "query": "연녹잠 누에고치가 콜레스테롤을 낮춘다는 연구 결과가 있어?",
        "answer_doc_ids": [5]
    },
    {
        "query": "병에 강한 고추 품종 연구 내용 요약해줘",
        "answer_doc_ids": [7, 8, 9, 10, 11, 12]
    },
    {
        "query": "고추 탄저병이나 풋마름병을 견디는 유전자원이 무엇인지 알려줘",
        "answer_doc_ids": [7, 8, 9, 10, 11, 12]
    },
    {
        "query": "간척지 구조물 안전 시공을 위한 복합기둥 공법을 설명해줘",
        "answer_doc_ids": [13, 14, 15, 16]
    },
    {
        "query": "복합기둥 공법이 기존 공법보다 어떤 점이 유리한가?",
        "answer_doc_ids": [15, 16]
    },
    {
        "query": "잎들깨 숨들의 호흡기 건강 기능성을 알려줘",
        "answer_doc_ids": [17, 18, 19, 20]
    },
    {
        "query": "숨들 품종이 산업적으로 어떤 가치가 있는지 설명해줘",
        "answer_doc_ids": [19, 20]
    },
    {
        "query": "에어로겔 다겹보온커튼이 난방비를 줄이는 이유는?",
        "answer_doc_ids": [21, 22, 23, 24]
    },
    {
        "query": "복숭아 디지털 육종 기술을 설명해줘",
        "answer_doc_ids": [25, 26, 27, 28]
    },
    {
        "query": "납작 복숭아(반도)의 육종 과정이 어떻게 효율화되었는지 알려줘",
        "answer_doc_ids": [27, 28]
    }
]

#### ② Recall@K 계산

In [36]:
def recall_at_k(retriever, test_cases, k=5):
    correct = 0
    total = len(test_cases)

    for case in test_cases:
        retrieved = retrieve_ids(retriever, case["query"], k=k)
        if any(doc_id in retrieved for doc_id in case["answer_doc_ids"]):
            correct += 1

    return correct / total

#### ③ Hybrid / Dense / Sparse 비교 실험

In [37]:
def debug_recall(retriever, test_cases, k=5):
    correct = 0
    total = len(test_cases)

    for case in test_cases:
        query = case["query"]
        gt_ids = case["answer_doc_ids"]
        
        # 모델 예측 (retrieved IDs)
        retrieved_docs = retriever.invoke(query)
        pred_ids = [d.metadata.get("id") for d in retrieved_docs[:k]]

        # 정답 여부 확인
        hit = any(doc_id in pred_ids for doc_id in gt_ids)
        if hit:
            correct += 1

        # ===== 디버깅 출력 =====
        print(f"\n[질문] {query}")
        print(f"- 예측 Top-{k} IDs: {pred_ids}")
        print(f"- 정답 IDs: {gt_ids}")
        print(f"- 정답 여부: {'O' if hit else 'X'}")
        print("-" * 50)

    print(f"\n총 Recall@{k}: {correct / total:.3f}")
debug_recall(retriever, test_cases, k=5)


[질문] 연녹잠 누에고치의 기능성과 활용 가능성을 알려줘
- 예측 Top-5 IDs: [6, 5, 4, 23]
- 정답 IDs: [4, 5, 6]
- 정답 여부: O
--------------------------------------------------

[질문] 연녹잠 누에고치가 콜레스테롤을 낮춘다는 연구 결과가 있어?
- 예측 Top-5 IDs: [5, 6, 4, 9]
- 정답 IDs: [5]
- 정답 여부: O
--------------------------------------------------

[질문] 병에 강한 고추 품종 연구 내용 요약해줘
- 예측 Top-5 IDs: [7, 10, 9, 13]
- 정답 IDs: [7, 8, 9, 10, 11, 12]
- 정답 여부: O
--------------------------------------------------

[질문] 고추 탄저병이나 풋마름병을 견디는 유전자원이 무엇인지 알려줘
- 예측 Top-5 IDs: [9, 7, 10, 12]
- 정답 IDs: [7, 8, 9, 10, 11, 12]
- 정답 여부: O
--------------------------------------------------

[질문] 간척지 구조물 안전 시공을 위한 복합기둥 공법을 설명해줘
- 예측 Top-5 IDs: [13, 14, 16, 15]
- 정답 IDs: [13, 14, 15, 16]
- 정답 여부: O
--------------------------------------------------

[질문] 복합기둥 공법이 기존 공법보다 어떤 점이 유리한가?
- 예측 Top-5 IDs: [14, 26, 13, 15]
- 정답 IDs: [15, 16]
- 정답 여부: O
--------------------------------------------------

[질문] 잎들깨 숨들의 호흡기 건강 기능성을 알려줘
- 예측 Top-5 IDs: [17, 19, 20, 6]
- 정답 IDs: [17, 18, 19

In [38]:
#print("Dense Recall@5:", recall_at_k(dense_retriever, test_cases, k=5))
#print("Sparse Recall@5:", recall_at_k(sparse_retriever, test_cases, k=5))
print("Hybrid Recall@5:", recall_at_k(hybrid_rrf, test_cases, k=5))

Hybrid Recall@5: 1.0


**관찰 포인트**

- Hybrid가 대부분 Recall에서 우수함  
- Sparse-only는 특정 키워드 문서에 강함  
- Dense-only는 표현이 달라진 경우에 강함  

### 6.3 Generation Evaluation (생성 품질 평가)

Generation은 다음 기준으로 평가한다.

✔ 1) Faithfulness (근거 기반 여부)  
- 답변이 문서에서 실제로 근거를 가지고 있는가?  
- 문서에 없는 내용을 hallucination으로 만들어내지 않았는가?

✔ 2) Relevance (질문과의 관련성)  
- 질문의 핵심을 정확히 답했는가?

✔ 3) Completeness  
- 문서에 있는 답을 빠짐없이 요약했는가?

✔ 4) Conciseness  
- 불필요하게 장황하지 않고 필요한 내용만 말했는가?

✔ 5) Citation Quality  
- 출처 인용이 정확한가?  
- 문서 번호/페이지 등 기반 정보가 연결되었는가?

#### Generation Evaluation 실습: LLM을 활용한 Generation 평가 (LLM-as-a-Judge)

LLM에게 “생성된 답변의 품질”을 직접 평가하게 할 수 있습니다.

In [39]:
judge_prompt = """
다음은 RAG 모델이 생성한 답변과, 참조 문서(Context)입니다.
답변이 문서에 근거했는지 0~1 점수로 평가하세요.

- 1점: 문서에서 근거를 명확히 가지고 있음
- 0점: 문서에 없는 내용을 생성함 (hallucination)
- 0.5점: 부분적으로만 근거가 있음

[질문]
{query}

[답변]
{answer}

[문맥]
{context}

점수만 출력하세요.
"""

from langchain_core.prompts import PromptTemplate
judge_template = PromptTemplate.from_template(judge_prompt)

def evaluate_answer(query, answer, context):
    judge = llm.invoke(judge_template.format(query=query, answer=answer, context=context))
    return float(judge.content.strip())

In [40]:
query = "농업 기술 보급 기준 알려줘"

docs = hybrid_rrf.get_relevant_documents(query)
context = format_context_with_ids(docs)

answer = llm.invoke(prompt.format(query=query, context=context)).content

score = evaluate_answer(query, answer, context)

print("=== 답변 ===")
print(answer)

print("\n=== Faithfulness Score ===")
print(score)

ValueError: could not convert string to float: '1점'

### 6.4 실무에서 사용하는 전체 평가 전략

기업이나 기관 등 대규모 환경에서는 다음과 같은 평가 절차를 종합적으로 수행합니다. 

1. **Retrieval 성능 평가**
   - Recall@K  
   - nDCG(순위 기반 품질 척도)  
   - Multi-query / Hybrid Retrieval 비교 분석  

2. **Generation 품질 평가**
   - LLM 기반 자동 평가(LLM-as-a-Judge)  
   - Human evaluation  
   - 생성된 답변의 근거 출처(citation) 검증  

3. **A/B 테스트**
   - 서로 다른 RAG 구성 중 어느 쪽이 더 우수한지 실험적으로 비교  

4. **Feedback Loop 구축**
   - 잘못된 답변 자동 수집  
   - 쿼리·문서 튜닝 및 재학습을 통한 지속적 개선

### 6.5 정리

1. **RAG 성능을 지속적으로 개선하는 유일한 방법은 체계적인 평가(evaluation)입니다.**  
2. Retrieval 평가에서는 문서 회수 능력을 나타내는 Recall@K가 가장 중요한 지표입니다.  
3. Generation 평가에서는 답변이 실제 문서에 근거하는지를 판단하는 Faithfulness가 핵심 기준입니다.  
4. LLM을 채점자(judge)로 활용하는 자동화된 평가 방식이 현재 가장 널리 사용됩니다.  
5. 이러한 평가 절차는 RAG 파이프라인을 비교·개선하고 품질을 안정적으로 높이는 데 필수 요소입니다.